In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"


# DATA_PATH = 'data/'
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [3]:
store_list = pd.read_excel(f"{DATA_PATH}요기요가게취합(0908업데이트).xlsx")


In [4]:
ygy_clean_0905 = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean.csv")


yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908.csv") #37000개


sentiment_train = pd.read_csv(f"{DATA_PATH}posneg_train_37000.csv")

neg_train = pd.read_csv(f"{DATA_PATH}gpt_neg_9300.csv")


train= yogiyo[['고객리뷰','사장댓글','별점']]

# train = yogiyo[['review', 'reply','rating']]

rename_cols = ['review', 'reply','rating']
train.columns = rename_cols
train.head()


tmp = sentiment_train[['sentiment','score']]
train = pd.concat([train,tmp],axis=1)

train

,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5,1,0.608408
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5,1,0.381793
...,...,...,...,...,...
37135,맛있어요 ㅎ저번에주문하고 맛있어서또주문핰건데\n늦은시간에시켜서그렄지 국물이 엄청 걸...,고객님 안녕하세요 짬뽕관입니다~~^-^\n소중한 리뷰 너무 감사합니다!!🙂\n재주문...,5,1,0.424215
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다!!^_^\n소중한 리뷰 남겨주셔 서 정말 달감사합니다!...,5,1,0.720691
37137,정말 맛있게 잘먹었어요!!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~~^_^\n맛있게 드셨다는 말씀에 힘이 나는거 같습니...,5,1,0.833007
37138,와우.. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 ...,고객님 안녕하세요 짬뽕관입니다!!^_^\n맛있게 드셔주시고 정성스러운 사진 리뷰까지...,5,1,0.900031


In [5]:
train = train[['review','reply','sentiment','score']]

In [6]:
neg_train

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585
...,...,...,...,...,...,...,...
9318,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9319,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [7]:
neg_train = neg_train[['gpt_review','gpt_reply','sentiment','score']]
neg_train.columns = train.columns

In [8]:
train_ft = pd.concat([train,neg_train], axis=0)
train_ft #총 46463개

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,1,0.608408
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,1,0.381793
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [9]:
train = train_ft

# 데이터 정제

- 정규표현식

In [10]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^가-힣0-9.,?!]") # 영어 제거
        result = pat.sub(" ", review)

        result = re.sub(r'^[.]\s*', '', result, count=1)
        result = re.sub(r'^[. ]+', '', result, count=1)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,1,0.608408
3,.,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,1,0.381793


In [11]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^가-힣0-9.,?!]") # 영어 제거
        result = pat.sub(" ", reply)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^[.]\s*', '', result, count=1)
        result = re.sub(r'^[. ]+', '', result, count=1)

        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1,0.608408
3,.,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,1,0.381793


- 한글이 포함되지 않은 리뷰 제거

In [12]:
pattern = r'^[^가-힣]*$'
# 패턴을 포함하는 문자열을 찾아 mask 생성
mask = (train['review'].str.match(pattern)) | (train['reply'].str.match(pattern))

train = train[~mask]
train

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,1,0.381793
5,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께 해주셔서 영광입니다. 다음에도 고객님의 즐거운 순...,1,0.549870
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


- 고객 id처리

In [13]:
# 님 앞의 문자들 고객님으로 처리. 짧은 리뷰 제거 이전 처리함.
train['reply'] = train['reply'].str.replace(r'.*님', '고객님', regex=True)

# for review in train['reply']:
#   if '고객님' in review:
#       print(review)
train

<ipython-input-13-e4dd164777d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = train['reply'].str.replace(r'.*님', '고객님', regex=True)


,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793
5,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님들께 더 맛있는 음식을 제공하기 위해 항상 노력하고 있습니다. 맛과 냄새에 대...,0,0.933854


- 짧은 리뷰 제거

In [14]:
mask = train['review'].str.len() <= 5
train[mask]

train = train[~mask]
train

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793
5,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님들께 더 맛있는 음식을 제공하기 위해 항상 노력하고 있습니다. 맛과 냄새에 대...,0,0.933854


In [15]:
mask = train['reply'].str.len() <= 16
train[mask]

train = train[~mask]
train

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793
5,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님들께 더 맛있는 음식을 제공하기 위해 항상 노력하고 있습니다. 맛과 냄새에 대...,0,0.933854


In [16]:
mask = train['sentiment'] == 0
neg_train = train[mask]

In [17]:
mask = train['sentiment'] == 1
pos_train = train[mask]

In [18]:
shuffled_pos_train = pos_train.sample(frac=1, random_state=42)  # random_state는 랜덤 시드값을 지정합니다.

In [19]:
pos_df = shuffled_pos_train[:len(neg_train)]

In [20]:
train_ft = pd.concat([neg_train, pos_df],axis=0)
train_ft

,review,reply,sentiment,score
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321
28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 다시 ...,0,0.685938
48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316
65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날도시락의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다...,0,0.829625
...,...,...,...,...
29792,연탄구이 비빔밥 맛있네요 !,식사 맛있게 하셨어요 !?! 언제든지 찾아주셔도 부족함 없이 드실 수 있도록 야채 ...,1,0.474923
30174,손목 아파서 아무것도 못할때 이 가게 없었으면 굶을뻔 짜글이 국물 남겨놓고 다음날 ...,고객님들의 원픽 맛집이 되기위해 최선을 다하고 있답니다! 언제든지 믿고 찾아주시면 ...,1,0.735911
14014,떡볶이 치킨 다 아주 맛있어요. 잘 먹었습니다.,고객님 덕에 너무 힘이 나네요 또 이용 해 주실거죠 ? 주말 잘 보내세요! 이용 해...,1,0.691521
27868,큼직큼직 굿굿 잘 먹었습니다,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,1,0.546858


In [21]:
train = train_ft

## 가게/음식/지점명 처리
--> 메뉴명 --> 지점명 --> 가게명 순으로 처리함

### 메뉴명 처리

- 요기요 메뉴명 칼럼 활용

In [22]:
ygy_clean_0905['food'] = ygy_clean_0905['food'].astype(str)

In [23]:
store_keyword = ['세요','니다','후기','조리','비조리','사이즈','가득','봉지','추천','감사','행사','이벤트','약속','사장','해요','주문','오픈','청년','추가','배달','서비스','배불','존맛','인기','최고','연인','다시']

for i in range(len(ygy_clean_0905['food'])):
    for keyword in store_keyword:
        if keyword in ygy_clean_0905['food'][i]:
            ygy_clean_0905['food'][i] = ''
            break  # 키워드가 하나라도 발견되면 루프 종료


<ipython-input-23-84f83caa8987>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ygy_clean_0905['food'][i] = ''


In [24]:
ygy_clean_0905

,review,reply,star,star_t,star_q,star_d,food,store
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제...,5,5,5,5.0,,1988응답하라추억의옛날도시락 방이점
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...,5,5,5,5.0,돼지고기김치찌개 대접밥 우삼겹된장찌개 대접밥 오색빛깔 잡채,1988응답하라추억의옛날도시락 방이점
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,안녕하세요 D 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기...,5,5,5,5.0,,1988응답하라추억의옛날도시락 방이점
3,맛있습니다 근데 국물이 안와서 아쉽습니다,안녕하세요 1988응답하라추억의옛날도시락입니다 저희 측 실수로 인하여 누락이 되었군...,4,4,4,3.0,양념갈비 옛날도시락,"1988응답하라추억의옛날도시락 방이점,1988응답"
4,맛있게잘먹었습니다,헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요,5,5,5,5.0,삼겹정찬도시락 돼지고기김치찌개 대접밥,1988응답하라추억의옛날도시락 방이점
...,...,...,...,...,...,...,...,...
24993,먹을만하다고느낍니다,놀부를 이용해 주셔서 감사합니다 큰 만족을 드리지 못한것 같아 죄송합니다 다만 다행...,4,3,3,3.0,nan,놀부 부대찌개 광주우산점
24994,조리를해야하는 불편함이 있긴했지만 매장에서 먹는맛 그대로이고 양도 엄청많네요 4인분...,저희 놀부를 이용해 주셔서감사드립니다 집에서 조리시에도 매장과 최대한 비슷하게 드실...,5,5,5,5.0,nan,놀부 부대찌개 광주우산점
24995,오랜만에 놀부부대찌게 맛있었는더 햄 양을 늘리심 좋을듯,놀부를 이용해 주셔서 감사합니다 맛있게 드셨다니 다행입니다 저희는 정해진 레시피가 ...,5,5,4,5.0,nan,놀부 부대찌개 광주우산점
24996,예전에 참전한 6 25 에서 먹은 부대찌개 맛이 나네요 맛있읍니다,저희 놀부부대찌개를 맛있게 드셔주셔서 감사합니다 어디서 드시더라도 항상 맛있는 음식...,5,5,5,5.0,nan,"놀부 부대찌개 광주우산점,대찌,부대찌개,부대찌"


In [25]:
ygy_food_list=[]

for food in ygy_clean_0905['food']:
    if isinstance(food, str):
        foods = food.split(' ')
        for food in foods :
            ygy_food_list.append(food)


In [26]:
def extract_korean(text):
    # 한글 문자만 추출하는 정규 표현식 사용
    korean_text = re.sub('[^가-힣]+', '', text)
    return korean_text

In [27]:
ygy_food_list = [extract_korean(text) for text in ygy_food_list if extract_korean(text) and len(extract_korean(text)) > 1]

ygy_food_list = set(ygy_food_list)
ygy_food_list = list(ygy_food_list)

print(len(ygy_food_list))
ygy_food_list

3732


['밀크티',
 '밀키스제로',
 '초코무스케이크',
 '무말랭이오징어젓갈양념반찬',
 '오븐스파게티',
 '통뼈닭발',
 '크림치즈볼개',
 '순대곱창',
 '어니언링',
 '펩시제로라임',
 '자성당',
 '오레오마카롱',
 '피치우롱스위티',
 '김치돼지목살',
 '델리만쥬',
 '옥수콘스무디',
 '알밥',
 '칸반찬',
 '참치회덮밥',
 '까이텃',
 '싸이플렉스버거',
 '닭볶음탕',
 '바나나킥라떼',
 '녹차라떼',
 '콩빵',
 '진심초밥',
 '프레첼',
 '떡형세트',
 '튀김',
 '자성당김밥',
 '수비드치킨로제파스탕',
 '여수',
 '옥수수',
 '동지',
 '고소한',
 '얇은돈까스와',
 '갈릭소스',
 '오징어튀김',
 '크레이지',
 '부타동',
 '시간의기다림',
 '써브웨이',
 '갈릭벌집껍데기',
 '공기밥',
 '팥빙수',
 '양은도시락',
 '체리콕',
 '통우럭매운탕',
 '오지치즈토스트',
 '통옥수수',
 '유자티',
 '도야볶음밥',
 '버팔로봉',
 '돈코츠라멘세트',
 '커플세트',
 '나홀로세트',
 '매운양념',
 '으뜸꼬마만두당',
 '캘리포니아롤',
 '커큐민',
 '맛있는',
 '포장',
 '고로케',
 '매콤벌집껍볶이',
 '달달한',
 '찹쌀치킨스틱',
 '오렌지',
 '콩나물뼈찜',
 '고기짬뽕',
 '빅스타',
 '울트라',
 '아메리칸핫도그',
 '카페라떼',
 '얼음캔',
 '오레오뚱카롱',
 '고구마치즈고로케',
 '불총육회비빔밥',
 '꽈리꼬추',
 '순대곱창인',
 '훈제치킨',
 '곱빼기',
 '치키치키차카초코초코초',
 '단족',
 '꼬마물육회',
 '셀프',
 '화이트트러플크림',
 '고추기름',
 '뽕따프라페',
 '우삼겹폭탄',
 '등급생돼지짜글이',
 '껍질없는순대',
 '흑미밥',
 '디카페인',
 '김치부대찌개',
 '누들',
 '청양고추개',
 '크림치즈미포함',
 '믹스킹다방커피',
 '단품초밥',
 '마끼아또',
 '골라비빔밥',
 '소고기볶음밥',
 '마늘보쌈

In [28]:
len(ygy_food_list)

3732

- 형태소 분석 후 형용사 등 제외

In [29]:
!pip install kiwipiepy

In [30]:
!pip install konlpy


In [31]:
# !python3 -m pip install --upgrade pip


In [32]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [33]:
tokens = kiwi.analyze(ygy_food_list)

remove_words_menu = []
lst = ['V','J','E','X','M','I','S']

for token in tokens :
    word = token[0][0][0].form
    pos = token[0][0][0].tag
#   if pos in ('NNB','NNP','NR','NP','VV','VA','VX','VCP','VCN','MM', 'MA', 'XR','MAG','MAJ') and len(word) > 1 :
    # if pos in ('NR','NP','VV','VA','VX','VCP','VCN','MM', 'MA', 'XR','MAG','MAJ') :
    if any(pos.startswith(prefix) for prefix in lst) :
        remove_words_menu.append(word) #NNG, NNP는 메뉴명 너무 많이 포함해서 처리가 불가함

remove_words_menu

# 2글자 고유명사 제외 품사 단어들은 제외 -> 3, 4 글자 이상은 가게명

['자',
 '고소하',
 '얇',
 '통',
 '맵',
 '맛있',
 '고로',
 '달달하',
 '단',
 '우',
 '단',
 '고르',
 '참',
 '순',
 '매콤',
 '화끈',
 '꾸',
 '매콤',
 '단',
 '시원하',
 '매콤',
 '오직',
 '듬뿍',
 '맵',
 '갈',
 '몽땅',
 '굽',
 '많이',
 '찌',
 '빠지',
 '탕',
 '달콤',
 '맛보',
 '든든',
 '두',
 '순',
 '전',
 '붉',
 '싸',
 '두',
 '맛있',
 '우',
 '비단',
 '굉장히',
 '생',
 '완전',
 '순',
 '순',
 '녹',
 '한',
 '참',
 '달',
 '얼큰',
 '탐나',
 '생',
 '만들',
 '아삭',
 '대박',
 '생',
 '납작',
 '진',
 '좋아하',
 '바삭바삭',
 '온',
 '뚝',
 '납작',
 '대',
 '매콤',
 '갓',
 '밀',
 '넣',
 '갓',
 '착',
 '몽글몽글',
 '생',
 '갓',
 '생',
 '매콤',
 '마',
 '완전',
 '로',
 '데리',
 '통통',
 '싱그럽',
 '만들',
 '약간',
 '따로',
 '땡',
 '꿀떡',
 '매콤',
 '구',
 '생',
 '휴',
 '좋',
 '빙',
 '생',
 '얼큰',
 '허',
 '블',
 '푸짐',
 '싱싱',
 '게',
 '동동',
 '새콤',
 '굽',
 '맵',
 '골고루',
 '체',
 '색다르',
 '매콤',
 '고르',
 '맵',
 '한',
 '세',
 '순',
 '보글보글',
 '진짜',
 '울',
 '위하',
 '초',
 '고소하',
 '오',
 '친',
 '혼자',
 '얼큰',
 '쫄깃',
 '노랗',
 '탕',
 '단',
 '매콤',
 '아',
 '두',
 '있',
 '생',
 '듬뿍',
 '삶',
 '쪼',
 '통',
 '싱글',
 '송송',
 '노랗',
 '갓',
 '로',
 '탄탄',
 '매콤',
 '맑',
 '생',
 '따로',
 '매콤',
 '생',
 '얼큰',
 '말',
 '두

In [34]:
filtered_ygy = [menu for menu in ygy_food_list if not any(word in menu for word in remove_words_menu) ]
print(len(filtered_ygy))
filtered_ygy

868


['옥수콘스무디',
 '바나나킥라떼',
 '콩빵',
 '프레첼',
 '튀김',
 '여수',
 '옥수수',
 '시간의기다림',
 '공기밥',
 '양은도시락',
 '도야볶음밥',
 '매운양념',
 '커큐민',
 '포장',
 '콩나물뼈찜',
 '빅스타',
 '얼음캔',
 '꽈리꼬추',
 '곱빼기',
 '셀프',
 '뽕따프라페',
 '흑미밥',
 '누들',
 '공기밥개포함',
 '간장덮밥',
 '무뼈닭발',
 '딸기라떼',
 '볶음짬뽕덮밥',
 '양쥬식',
 '과일티',
 '뽈살수육',
 '사천',
 '바질커리',
 '미니장국',
 '스팸옛날도시락',
 '제사음식패키지',
 '꿀쌈장',
 '누룽지연유라떼',
 '탠탠',
 '술국',
 '키위',
 '닭껍질',
 '미니간장',
 '왕판타스틱',
 '버터바',
 '파삼파삼',
 '공기밥개',
 '돌돔회',
 '기름장',
 '밥도둑',
 '바닐라라떼',
 '물냉면',
 '공심채볶음',
 '국물',
 '간장찜닭',
 '씬바샤삭',
 '폴트',
 '플러스',
 '도야짬뽕',
 '찰흑미밥',
 '산뽕잎',
 '닭다리',
 '삼선짬뽕밥',
 '사워',
 '뚱캔',
 '슈스틱',
 '트리플셋트',
 '궁합',
 '콘샐러드',
 '사과',
 '셀프주먹밥',
 '리얼키위쥬스',
 '삼선볶음밥',
 '퍼포먼스',
 '담짠담짠',
 '코코넛음료',
 '믹스너',
 '해물쌀국수',
 '샐러드',
 '찜무침',
 '야채죽',
 '선지국',
 '브루',
 '천연',
 '삼겹',
 '맛중독',
 '도시락',
 '농협',
 '계란부침덮밥',
 '제육김밥',
 '훈제',
 '홍콩',
 '캬멜바난',
 '수박쥬스',
 '칠성',
 '청춘',
 '스팸폭탄',
 '짬뽕',
 '모닝빵',
 '뉴욕',
 '맛집랭킹',
 '유부',
 '와플',
 '모듬튀김',
 '북경',
 '와플토핑',
 '감바스',
 '페스토',
 '밥도둑사리배',
 '모둠튀김',
 '플레인요거트',
 '놀부사리',
 '꽈리',
 '해물의',
 '드래곤',
 '육사시미',
 '벌집',
 '

- 2차 거르기

In [35]:
add_remove_list = [
    "메뉴", "궁함", "싹쓰리", "중독", "찰떡", "형제", "곱배기", "출연", "압도적", "별도",
    "음료", "선택", "메뉴", "실속", "춘천식", "각개", "얼음", "삼색", "바디", "셀프",
    "랭킹", "모듬", "정식", "혼밥", "플러스", "굿모닝", "스틱", "명품", "라떼는", "매력",
    "토핑", "배터짐", "뜸들인", "밥도둑", "입문", "추억", "명품", "미니", "셀프", "꿀맛",
    "양념", "요리", "클럽", "시간", "기다림", "조각", "요거봐라", "해장", "무도", "다시",
    "유부", "옛날", "모듬", "주의", "들을", "골라먹는", "정석", "박스", "재료", "청년",
    "맛중독", "중독", "특제", "임산부", "맛집랭킹", "조린", "판매량", "무국적", "할인",
    "포함", "시간의 정성", "미포함", "특선",'궁합', '포장','배달','맛','양','가격','서비스',
    '이벤트',"셋트", "매운", '맛집','의성','숙성시킨','폭탄', '특상품','판매','천년','해주는',
    '씨없는','매워요','다시','주인장','주인','다시']


In [36]:
filtered_ygy = [menu for menu in filtered_ygy if not any(word in menu for word in add_remove_list) ]
print(len(filtered_ygy))

filtered_ygy = set(filtered_ygy)
filtered_ygy = list(filtered_ygy)
filtered_ygy

691


['플레인요거트스무디',
 '지삼선',
 '쫀득꼬들',
 '리터',
 '벌꿀라떼',
 '물육회',
 '딸기주스',
 '숙회',
 '냉면육수',
 '옥수콘스무디',
 '찜닭무',
 '인눈꽃냉면',
 '바나나킥라떼',
 '케찹',
 '월급날엔',
 '콩빵',
 '프레첼',
 '튀김',
 '켄터키',
 '여수',
 '청도홍시주스',
 '스위트',
 '옥수수',
 '쳥년',
 '해물바질덮밥',
 '쌈장',
 '제육볶음',
 '공기밥',
 '딸기바나나',
 '토종',
 '잡채',
 '스팸',
 '도야볶음밥',
 '재래',
 '콜드브루라떼',
 '해물핫바',
 '상무식혜',
 '회관',
 '커큐민',
 '스팸장',
 '연유라떼',
 '프라페',
 '콩나물뼈찜',
 '빅스타',
 '꽈리꼬추',
 '완숙',
 '돼지국밥',
 '곱빼기',
 '혼술',
 '뽕따프라페',
 '캘리포니안',
 '정육왕',
 '흑미밥',
 '브리또',
 '라즈지',
 '쉑쉑',
 '누들',
 '인용',
 '수육',
 '와규육회',
 '화채',
 '육연물회',
 '간장덮밥',
 '조미김',
 '캬라멜',
 '쁘링클',
 '넛츠브릭',
 '커스타드',
 '무뼈닭발',
 '딸기라떼',
 '매실티',
 '후무스',
 '갸또',
 '벌교꼬막무침',
 '인절미',
 '플레인',
 '볶음짬뽕덮밥',
 '봉골레',
 '삼선짬뽕죽',
 '청포도',
 '충북등',
 '뽈살수육',
 '과일티',
 '개입',
 '안심',
 '해산물',
 '석쇠',
 '사천',
 '바질커리',
 '공깃밥',
 '그라운드룰',
 '중국',
 '닭목살덮밥',
 '무침',
 '그린티라떼',
 '웨지',
 '누텔라토스트',
 '황궁볶음밥',
 '제사음식패키지',
 '꿀쌈장',
 '야채핫바',
 '필라프위',
 '주스',
 '누룽지연유라떼',
 '탠탠',
 '특계란찜',
 '칼국수',
 '백숙죽',
 '라떼',
 '술국',
 '일태리',
 '투명콜라',
 '무화과',
 '노란',
 '깻잎',
 '핫봉',
 '분식집',
 '된장',
 '핫윙스',


In [37]:
remove_words_menu = remove_words_menu + add_remove_list

remove_words_menu = set(remove_words_menu)
remove_words_menu = list(remove_words_menu)
remove_words_menu

['쫄깃',
 '보다',
 '꿀떡',
 '의성',
 '작',
 '울',
 '굽',
 '싱싱',
 '쫄깃쫄깃',
 '밥도둑',
 '불',
 '주인',
 '라떼는',
 '몽땅',
 '체',
 '보글보글',
 '내',
 '양',
 '달',
 '간단',
 '쏠',
 '비단',
 '궁함',
 '통통',
 '담백하',
 '아삭',
 '박스',
 '청년',
 '착',
 '족',
 '서비스',
 '허',
 '특상품',
 '야들야들',
 '튀기',
 '셋트',
 '진짜',
 '푸짐',
 '각개',
 '많이',
 '강추',
 '그냥',
 '만드',
 '게',
 '바싹',
 '위하',
 '납작',
 '양념',
 '한',
 '상큼',
 '싱그럽',
 '플러스',
 '파이',
 '맛나',
 '형제',
 '엄청',
 '굿모닝',
 '임산부',
 '대박',
 '주의',
 '앵',
 '알리',
 '정석',
 '조각',
 '못',
 '부드럽',
 '직접',
 '아구',
 '포장',
 '바디',
 '잔뜩',
 '땡',
 '달콤',
 '꿀맛',
 '블',
 '본',
 '따뜻하',
 '차',
 '매콤하',
 '탱글탱글',
 '풋풋',
 '참',
 '알',
 '힘차',
 '꼬들꼬들',
 '두',
 '복',
 '시간의 정성',
 '맵',
 '바삭바삭',
 '온',
 '혼자',
 '보들보들',
 '소',
 '알차',
 '가',
 '실속',
 '셀프',
 '뜨끈',
 '혼밥',
 '매일',
 '까',
 '배달',
 '원',
 '촉촉',
 '반',
 '따로',
 '덜',
 '밀',
 '궁합',
 '최',
 '찰떡',
 '미포함',
 '휴',
 '주인장',
 '진',
 '진하',
 '매운',
 '빠지',
 '라지',
 '시원하',
 '통',
 '달달하',
 '크',
 '정식',
 '맛있',
 '얇',
 '세',
 '이',
 '어',
 '출연',
 '넙적',
 '든든',
 '시간',
 '아이',
 '무도',
 '쉬',
 '명품',
 '탄탄',
 '환',
 '메뉴',
 '중독',
 '좋',
 '몬',
 '해장

In [38]:
# remove_words_menu
# add_remove = ['부산','사장님추천','라떼는','정석','매력적인','해주는','익혀드셔야해요','데워드셔야','바디감','부드러운','판매량위','시간','주문','조합','맛집랭킹','혼밥끝판왕','배터짐주의','혼술','월급날엔','사장','시간의정성','부탁드립니다','추억의','분정도','']

- 음식 표준 데이터 활용

In [39]:
food_list = pd.read_csv(f'{DATA_PATH}전국통합식품영양성분정보(음식)표준데이터.csv', encoding='cp949')

In [40]:
food_list = set(food_list['대표식품명'].tolist())

In [41]:
food_list = list(food_list)
len(food_list)

758

In [42]:
food_list

['깻잎김치',
 '마카롱',
 '밀크티',
 '참나물',
 '호박죽',
 '닭다리구이',
 '어묵국(어묵탕)',
 '소갈비찜',
 '닭죽',
 '순대',
 '치즈볼',
 '경단',
 '매실장아찌',
 '숙회',
 '꼬막무침',
 '콩나물국밥',
 '버섯전골',
 '수수경단',
 '알밥',
 '견과',
 '참치회덮밥',
 '닭볶음탕',
 '감자탕',
 '꼬막찜',
 '소시지볶음',
 '오이소박이',
 '닭모래집볶음',
 '표고버섯볶음',
 '애호박전',
 '복찜',
 '옥수수',
 '가지나물',
 '산적',
 '울면',
 '어탕',
 '곤드레나물밥',
 '모듬찰떡',
 '밴댕이무침',
 '송편',
 '완자전',
 '지리탕',
 '잡채',
 '감자튀김',
 '굴무침',
 '어육소시지전',
 '소고기전골',
 '돼지고기볶음',
 '치즈빵',
 '캘리포니아롤',
 '기장밥',
 '효모빵',
 '냉이나물',
 '조기찜',
 '닭볶음(닭갈비)',
 '꼬치구이',
 '멸치젓',
 '배추전',
 '시래기나물',
 '대구찜',
 '해장국',
 '얼갈이배추김치',
 '버섯조림',
 '채소튀김',
 '돼지갈비찜',
 '복숭아홍차',
 '허브차',
 '피자',
 '홍어무침',
 '흑미밥',
 '녹두빈대떡',
 '대구볼찜',
 '낙지전골',
 '만두전골',
 '수육',
 '닭다리튀김',
 '곰탕',
 '대추차',
 '배추국',
 '쥐포튀김',
 '우롱차',
 '조개젓',
 '호박찌개',
 '호박볶음',
 '돼지고기볶음(제육볶음)',
 '풋고추찜',
 '연어롤',
 '스파게티',
 '인절미',
 '생강차',
 '떡국',
 '탄산음료',
 '낙지무침',
 '해물볶음',
 '부추김치',
 '하이라이스',
 '장어탕',
 '골뱅이무침',
 '떡만두국',
 '푸딩',
 '고추전',
 '카레소스',
 '절편',
 '치즈',
 '잔멸치볶음',
 '황태해장국',
 '오리고기구이',
 '콩밥',
 '삼치구이',
 '연근조림',
 '냉채',
 '새송이버섯전',
 '김치찌개',
 '건새우볶음',
 '

In [43]:
ygy_food_list = ['양식','일식','중식','중국집','태국음식','한식','분식','도시락','카레','커피','국밥','마라탕','마라샹궈','우동','나가사키','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','짬뽕','탕수육','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
             '새우','돼지','닭','찌개','밥','타코야키',
                 '양파','숙주','고수'] # 재료명 제거

In [44]:
# menu_name = food_list + ygy_food_list + filtered_ygy

menu_name = food_list + ygy_food_list

In [45]:
#한글자 삭제
for i in menu_name:
    if len(i) < 2 :
        print(i)
        menu_name.remove(i)

밤
번
닭
밥


In [46]:
menu_name = set(menu_name)
menu_name = list(menu_name)
print(len(menu_name))
menu_name

795


['깻잎김치',
 '마카롱',
 '밀크티',
 '참나물',
 '호박죽',
 '닭다리구이',
 '어묵국(어묵탕)',
 '소갈비찜',
 '닭죽',
 '순대',
 '치즈볼',
 '경단',
 '매실장아찌',
 '숙회',
 '꼬막무침',
 '콩나물국밥',
 '버섯전골',
 '수수경단',
 '알밥',
 '견과',
 '참치회덮밥',
 '닭볶음탕',
 '감자탕',
 '꼬막찜',
 '소시지볶음',
 '오이소박이',
 '닭모래집볶음',
 '표고버섯볶음',
 '애호박전',
 '복찜',
 '옥수수',
 '가지나물',
 '산적',
 '울면',
 '어탕',
 '곤드레나물밥',
 '모듬찰떡',
 '밴댕이무침',
 '송편',
 '완자전',
 '지리탕',
 '잡채',
 '감자튀김',
 '굴무침',
 '어육소시지전',
 '소고기전골',
 '돼지고기볶음',
 '치즈빵',
 '캘리포니아롤',
 '조기찜',
 '기장밥',
 '효모빵',
 '냉이나물',
 '닭볶음(닭갈비)',
 '꼬치구이',
 '멸치젓',
 '고수',
 '배추전',
 '일식',
 '시래기나물',
 '대구찜',
 '해장국',
 '얼갈이배추김치',
 '버섯조림',
 '채소튀김',
 '돼지갈비찜',
 '복숭아홍차',
 '허브차',
 '피자',
 '홍어무침',
 '흑미밥',
 '녹두빈대떡',
 '중국집',
 '대구볼찜',
 '누들',
 '낙지전골',
 '만두전골',
 '수육',
 '닭다리튀김',
 '곰탕',
 '대추차',
 '배추국',
 '쥐포튀김',
 '우롱차',
 '조개젓',
 '고기',
 '호박찌개',
 '호박볶음',
 '돼지고기볶음(제육볶음)',
 '풋고추찜',
 '연어롤',
 '스파게티',
 '인절미',
 '생강차',
 '떡국',
 '탄산음료',
 '낙지무침',
 '해물볶음',
 '부추김치',
 '하이라이스',
 '장어탕',
 '골뱅이무침',
 '떡만두국',
 '푸딩',
 '고추전',
 '카레소스',
 '절편',
 '치즈',
 '잔멸치볶음',
 '황태해장국',
 '오리고기구이',
 '콩밥',
 '삼치구이',
 '연근조림',
 '냉

- 일반 명사 등 제외

In [47]:
remove_words_menu = [item for item in remove_words_menu if len(item) > 1]

In [48]:
len(remove_words_menu)

200

In [49]:
menu_name

['깻잎김치',
 '마카롱',
 '밀크티',
 '참나물',
 '호박죽',
 '닭다리구이',
 '어묵국(어묵탕)',
 '소갈비찜',
 '닭죽',
 '순대',
 '치즈볼',
 '경단',
 '매실장아찌',
 '숙회',
 '꼬막무침',
 '콩나물국밥',
 '버섯전골',
 '수수경단',
 '알밥',
 '견과',
 '참치회덮밥',
 '닭볶음탕',
 '감자탕',
 '꼬막찜',
 '소시지볶음',
 '오이소박이',
 '닭모래집볶음',
 '표고버섯볶음',
 '애호박전',
 '복찜',
 '옥수수',
 '가지나물',
 '산적',
 '울면',
 '어탕',
 '곤드레나물밥',
 '모듬찰떡',
 '밴댕이무침',
 '송편',
 '완자전',
 '지리탕',
 '잡채',
 '감자튀김',
 '굴무침',
 '어육소시지전',
 '소고기전골',
 '돼지고기볶음',
 '치즈빵',
 '캘리포니아롤',
 '조기찜',
 '기장밥',
 '효모빵',
 '냉이나물',
 '닭볶음(닭갈비)',
 '꼬치구이',
 '멸치젓',
 '고수',
 '배추전',
 '일식',
 '시래기나물',
 '대구찜',
 '해장국',
 '얼갈이배추김치',
 '버섯조림',
 '채소튀김',
 '돼지갈비찜',
 '복숭아홍차',
 '허브차',
 '피자',
 '홍어무침',
 '흑미밥',
 '녹두빈대떡',
 '중국집',
 '대구볼찜',
 '누들',
 '낙지전골',
 '만두전골',
 '수육',
 '닭다리튀김',
 '곰탕',
 '대추차',
 '배추국',
 '쥐포튀김',
 '우롱차',
 '조개젓',
 '고기',
 '호박찌개',
 '호박볶음',
 '돼지고기볶음(제육볶음)',
 '풋고추찜',
 '연어롤',
 '스파게티',
 '인절미',
 '생강차',
 '떡국',
 '탄산음료',
 '낙지무침',
 '해물볶음',
 '부추김치',
 '하이라이스',
 '장어탕',
 '골뱅이무침',
 '떡만두국',
 '푸딩',
 '고추전',
 '카레소스',
 '절편',
 '치즈',
 '잔멸치볶음',
 '황태해장국',
 '오리고기구이',
 '콩밥',
 '삼치구이',
 '연근조림',
 '냉

In [50]:
indices_to_remove = []

for i, menu in enumerate(menu_name):
    for j in remove_words_menu:
        if j in menu:
            print(menu)  # 삭제할 메뉴 출력 (선택사항)
            indices_to_remove.append(i)  # 삭제할 요소의 인덱스 저장

# 삭제할 요소를 뒤에서부터 삭제
for index in reversed(indices_to_remove):
    if 0 <= index < len(menu_name):  # 유효한 인덱스 범위 내인 경우에만 삭제
        del menu_name[index]


곤드레나물밥
모듬찰떡
모듬찰떡
해장국
탄산음료
황태해장국
도라지나물무침
기타음료
다시마무침
아구탕
도라지생채
다시마튀각
파이/만주
미꾸라지튀김
탄탄면
매운탕
스크램블드에그
꿀떡
치킨데리야끼
아이스티
치즈스틱
유부국
크로켓(고로케)
함박스테이크
아이스크림
찰떡빵
곤드레밥


In [51]:
menu_name

['깻잎김치',
 '마카롱',
 '밀크티',
 '참나물',
 '호박죽',
 '닭다리구이',
 '어묵국(어묵탕)',
 '소갈비찜',
 '닭죽',
 '순대',
 '치즈볼',
 '경단',
 '매실장아찌',
 '숙회',
 '꼬막무침',
 '콩나물국밥',
 '버섯전골',
 '수수경단',
 '알밥',
 '견과',
 '참치회덮밥',
 '닭볶음탕',
 '감자탕',
 '꼬막찜',
 '소시지볶음',
 '오이소박이',
 '닭모래집볶음',
 '표고버섯볶음',
 '애호박전',
 '복찜',
 '옥수수',
 '가지나물',
 '산적',
 '울면',
 '어탕',
 '송편',
 '완자전',
 '지리탕',
 '잡채',
 '감자튀김',
 '굴무침',
 '어육소시지전',
 '소고기전골',
 '돼지고기볶음',
 '치즈빵',
 '캘리포니아롤',
 '조기찜',
 '기장밥',
 '효모빵',
 '냉이나물',
 '닭볶음(닭갈비)',
 '꼬치구이',
 '멸치젓',
 '고수',
 '배추전',
 '일식',
 '시래기나물',
 '대구찜',
 '얼갈이배추김치',
 '버섯조림',
 '채소튀김',
 '돼지갈비찜',
 '복숭아홍차',
 '허브차',
 '피자',
 '홍어무침',
 '흑미밥',
 '녹두빈대떡',
 '중국집',
 '대구볼찜',
 '누들',
 '낙지전골',
 '만두전골',
 '수육',
 '닭다리튀김',
 '곰탕',
 '대추차',
 '배추국',
 '쥐포튀김',
 '우롱차',
 '조개젓',
 '고기',
 '호박찌개',
 '호박볶음',
 '돼지고기볶음(제육볶음)',
 '풋고추찜',
 '연어롤',
 '스파게티',
 '인절미',
 '생강차',
 '떡국',
 '낙지무침',
 '해물볶음',
 '부추김치',
 '하이라이스',
 '장어탕',
 '골뱅이무침',
 '떡만두국',
 '푸딩',
 '고추전',
 '카레소스',
 '절편',
 '치즈',
 '잔멸치볶음',
 '오리고기구이',
 '콩밥',
 '삼치구이',
 '연근조림',
 '냉채',
 '새송이버섯전',
 '김치찌개',
 '건새우볶음',
 '열무김치',
 '온면',
 '알탕',


In [52]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']

#위 일반명사에, 조사를 합친 경우도 제외함

In [53]:
remove_words_menu

['쫄깃',
 '보다',
 '꿀떡',
 '의성',
 '싱싱',
 '쫄깃쫄깃',
 '밥도둑',
 '주인',
 '라떼는',
 '몽땅',
 '보글보글',
 '간단',
 '비단',
 '궁함',
 '통통',
 '담백하',
 '아삭',
 '박스',
 '청년',
 '서비스',
 '특상품',
 '야들야들',
 '튀기',
 '셋트',
 '진짜',
 '푸짐',
 '각개',
 '많이',
 '강추',
 '그냥',
 '만드',
 '바싹',
 '위하',
 '납작',
 '양념',
 '상큼',
 '싱그럽',
 '플러스',
 '파이',
 '맛나',
 '형제',
 '엄청',
 '굿모닝',
 '임산부',
 '대박',
 '주의',
 '알리',
 '정석',
 '조각',
 '부드럽',
 '직접',
 '아구',
 '포장',
 '바디',
 '잔뜩',
 '달콤',
 '꿀맛',
 '따뜻하',
 '매콤하',
 '탱글탱글',
 '풋풋',
 '힘차',
 '꼬들꼬들',
 '시간의 정성',
 '바삭바삭',
 '혼자',
 '보들보들',
 '알차',
 '실속',
 '셀프',
 '뜨끈',
 '혼밥',
 '매일',
 '배달',
 '촉촉',
 '따로',
 '궁합',
 '찰떡',
 '미포함',
 '주인장',
 '진하',
 '매운',
 '빠지',
 '라지',
 '시원하',
 '달달하',
 '정식',
 '맛있',
 '출연',
 '넙적',
 '든든',
 '시간',
 '아이',
 '무도',
 '명품',
 '탄탄',
 '메뉴',
 '중독',
 '해장',
 '음료',
 '신선',
 '뺨치',
 '에그',
 '따듯하',
 '싹쓰리',
 '곤드레',
 '배터짐',
 '맛집',
 '좋아하',
 '동동',
 '토핑',
 '떠먹',
 '다시',
 '골라먹는',
 '새콤',
 '부들부들',
 '달달',
 '없이',
 '별도',
 '만들',
 '얼큰',
 '쑥쑥',
 '옛날',
 '매콤',
 '얼음',
 '맛중독',
 '쫀득쫀득',
 '굉장히',
 '모든',
 '약간',
 '아삭아삭',
 '추억',
 '특선',
 '판매',
 '입문',
 '스틱',

In [54]:
remove_withparticles = []

for word in remove_words_menu :
  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

- 메뉴명을 대체

In [55]:
menu_name

['깻잎김치',
 '마카롱',
 '밀크티',
 '참나물',
 '호박죽',
 '닭다리구이',
 '어묵국(어묵탕)',
 '소갈비찜',
 '닭죽',
 '순대',
 '치즈볼',
 '경단',
 '매실장아찌',
 '숙회',
 '꼬막무침',
 '콩나물국밥',
 '버섯전골',
 '수수경단',
 '알밥',
 '견과',
 '참치회덮밥',
 '닭볶음탕',
 '감자탕',
 '꼬막찜',
 '소시지볶음',
 '오이소박이',
 '닭모래집볶음',
 '표고버섯볶음',
 '애호박전',
 '복찜',
 '옥수수',
 '가지나물',
 '산적',
 '울면',
 '어탕',
 '송편',
 '완자전',
 '지리탕',
 '잡채',
 '감자튀김',
 '굴무침',
 '어육소시지전',
 '소고기전골',
 '돼지고기볶음',
 '치즈빵',
 '캘리포니아롤',
 '조기찜',
 '기장밥',
 '효모빵',
 '냉이나물',
 '닭볶음(닭갈비)',
 '꼬치구이',
 '멸치젓',
 '고수',
 '배추전',
 '일식',
 '시래기나물',
 '대구찜',
 '얼갈이배추김치',
 '버섯조림',
 '채소튀김',
 '돼지갈비찜',
 '복숭아홍차',
 '허브차',
 '피자',
 '홍어무침',
 '흑미밥',
 '녹두빈대떡',
 '중국집',
 '대구볼찜',
 '누들',
 '낙지전골',
 '만두전골',
 '수육',
 '닭다리튀김',
 '곰탕',
 '대추차',
 '배추국',
 '쥐포튀김',
 '우롱차',
 '조개젓',
 '고기',
 '호박찌개',
 '호박볶음',
 '돼지고기볶음(제육볶음)',
 '풋고추찜',
 '연어롤',
 '스파게티',
 '인절미',
 '생강차',
 '떡국',
 '낙지무침',
 '해물볶음',
 '부추김치',
 '하이라이스',
 '장어탕',
 '골뱅이무침',
 '떡만두국',
 '푸딩',
 '고추전',
 '카레소스',
 '절편',
 '치즈',
 '잔멸치볶음',
 '오리고기구이',
 '콩밥',
 '삼치구이',
 '연근조림',
 '냉채',
 '새송이버섯전',
 '김치찌개',
 '건새우볶음',
 '열무김치',
 '온면',
 '알탕',


In [56]:
def replace_menu_names(text):
    menu_replace_list = []
    for name1 in menu_name:
        if name1 in text:
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
            menu_replace_list.append(name1)
        else:
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                # if part == text[:len(part)]:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in remove_words_menu + remove_withparticles :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<menu>", text)
                        menu_replace_list.append(part)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text, menu_replace_list

In [57]:
replace_menu_names('라화방 마라탕에서 맛있게 식사했어요. 고기정식 진짜 너무 맛있음. 후식으로 주신 과일도요')

('라화방 <menu>에서 맛있게 식사했어요. <menu>정식 진짜 너무 맛있음. 후식으로 주신 <menu>도요',
 ['고기', '마라탕', '과일'])

In [58]:
empty_list = [ [] for _ in range(len(train)) ]
train['replaced_menu_names'] = empty_list

In [59]:
train = train.reset_index()
train

,index,review,reply,sentiment,score,replaced_menu_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 다시 ...,0,0.685938,[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날도시락의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다...,0,0.829625,[]
...,...,...,...,...,...,...
22079,29792,연탄구이 비빔밥 맛있네요 !,식사 맛있게 하셨어요 !?! 언제든지 찾아주셔도 부족함 없이 드실 수 있도록 야채 ...,1,0.474923,[]
22080,30174,손목 아파서 아무것도 못할때 이 가게 없었으면 굶을뻔 짜글이 국물 남겨놓고 다음날 ...,고객님들의 원픽 맛집이 되기위해 최선을 다하고 있답니다! 언제든지 믿고 찾아주시면 ...,1,0.735911,[]
22081,14014,떡볶이 치킨 다 아주 맛있어요. 잘 먹었습니다.,고객님 덕에 너무 힘이 나네요 또 이용 해 주실거죠 ? 주말 잘 보내세요! 이용 해...,1,0.691521,[]
22082,27868,큼직큼직 굿굿 잘 먹었습니다,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,1,0.546858,[]


In [60]:
for i, reply in tqdm(enumerate(train['reply']), desc="Processing Reply", ncols=100):
    text, menu_replace_list = replace_menu_names(reply)
    # 'reply' 칼럼에 대체된 리뷰 텍스트 저장
    train['reply'][i] = text

    # 'replaced_store_names' 칼럼에서 목록을 가져온 후, 새로운 목록을 추가
    # existing_list = train.at[i, 'replaced_store_names']
    train['replaced_menu_names'][i] = menu_replace_list


Processing Reply: 0it [00:00, ?it/s]

<ipython-input-60-e2bea9408aa6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = text
<ipython-input-60-e2bea9408aa6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['replaced_menu_names'][i] = menu_replace_list


In [61]:
train['replaced_menu_names'] = train['replaced_menu_names'].apply(set).apply(list)

In [62]:
mask = train['reply'].str.contains('<menu>')
train[mask]

,index,review,reply,sentiment,score,replaced_menu_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,[비스]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,[무나]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날<menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 ...,0,0.829625,[도시락]
7,91,맛은잇는데 청국장이넘짜요 글구 공기밥추가햇는데 안왓어요,"고객님의 환한 <menu>를 기대하며, 따뜻한 마음을 전달드릴 것을 약속드립니다 중...",0,0.759104,[미소]
8,92,맛이나 양은 무난해요 그런데. 배달예정시간이 무색하게 늦게도착 차라리 배달 예정시간...,1988응답하라추억의옛날<menu>을 선택해 주셔서 감사합니다. 저희 매장에서는 배...,0,0.877273,[도시락]
...,...,...,...,...,...,...
22075,36054,잘먹겠습니다 오늘도.,백소정 객사점 입니다 멋진 리뷰 너무 감사드립니다 항상 변치않고 좋은 음식으로 보답...,1,0.675867,[비스]
22077,23045,엄청 맛있는 편은 아니지만 꽤 먹을만합니다! 리뷰이벤트로 주신 마늘돈가스.흰 밥이랑...,소중한 리뷰 감사합니다 다음에는 <menu>돈까스 드셔보세용. 청주에는 저희 밖에 ...,1,0.416850,[마늘]
22079,29792,연탄구이 비빔밥 맛있네요 !,식사 맛있게 하셨어요 !?! 언제든지 찾아주셔도 부족함 없이 드실 수 있도록 <me...,1,0.474923,"[야채, 비빔, 고기]"
22082,27868,큼직큼직 굿굿 잘 먹었습니다,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,1,0.546858,"[초밥, 시리]"


In [63]:
mask = train['replaced_menu_names'].str.len() > 0
train[mask]

,index,review,reply,sentiment,score,replaced_menu_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,[비스]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,[무나]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날<menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 ...,0,0.829625,[도시락]
7,91,맛은잇는데 청국장이넘짜요 글구 공기밥추가햇는데 안왓어요,"고객님의 환한 <menu>를 기대하며, 따뜻한 마음을 전달드릴 것을 약속드립니다 중...",0,0.759104,[미소]
8,92,맛이나 양은 무난해요 그런데. 배달예정시간이 무색하게 늦게도착 차라리 배달 예정시간...,1988응답하라추억의옛날<menu>을 선택해 주셔서 감사합니다. 저희 매장에서는 배...,0,0.877273,[도시락]
...,...,...,...,...,...,...
22075,36054,잘먹겠습니다 오늘도.,백소정 객사점 입니다 멋진 리뷰 너무 감사드립니다 항상 변치않고 좋은 음식으로 보답...,1,0.675867,[비스]
22077,23045,엄청 맛있는 편은 아니지만 꽤 먹을만합니다! 리뷰이벤트로 주신 마늘돈가스.흰 밥이랑...,소중한 리뷰 감사합니다 다음에는 <menu>돈까스 드셔보세용. 청주에는 저희 밖에 ...,1,0.416850,[마늘]
22079,29792,연탄구이 비빔밥 맛있네요 !,식사 맛있게 하셨어요 !?! 언제든지 찾아주셔도 부족함 없이 드실 수 있도록 <me...,1,0.474923,"[야채, 비빔, 고기]"
22082,27868,큼직큼직 굿굿 잘 먹었습니다,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,1,0.546858,"[초밥, 시리]"


### 가게명 처리


In [64]:
ygy_store_list = store_list['가게이름'].tolist()

In [65]:
ygy_store_list

['1988응답하라추억의옛날도시락-방이점',
 '워커스커피 로스터스',
 '떡깨비-가락점',
 '당치땡-헬리오시티점',
 '우시장국밥-석촌점',
 '고돼지-송파점',
 '마라의신마라탕&마라샹궈',
 '춘리마라탕-송파점',
 '손수경의육회담은연어-가락점',
 '무국적식탁-송파점',
 '청년치킨-삼전점',
 '송파만다린',
 '배달의쌀국수-송파점',
 '닭강점-헬리오시티점',
 '토핑폭탄김치찜&김치찌개-송파점',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82-송파점',
 '더티베지',
 '잭아저씨족발보쌈-본점',
 '육회&연어-미아본점',
 '갈비민족-본점',
 '구구족-성신여대역점',
 '피자보이시나-대학로점',
 '왕빈자삼파전-미아본점',
 '행복한찜닭-성북점',
 '스시사소우',
 '저팔계&중국&마라탕&덮밥&요리&전문점',
 '명작파스타-성북점',
 '올데이파스타-동대문구점',
 '대한냉면-성북점',
 '인생제육',
 '고기듬뿍미트박스-성북점',
 '피자스웨그-성북점',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다-북서울꿈의숲점',
 '더팔당매운오징어&닭발 김치찜',
 '우리집반찬도시락-혜화막내딸지점',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비&마약볶음밥',
 '대찌 대파생고기김치찌개&정육왕국물두루치기',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜&김치찌개-부천점',
 '1989마라탕-부천점',
 '미친피자-부평점',
 '천년닭강정-부천부평점',
 '수상한삼겹살-부천점',
 '수제죽전문점-본가진죽-본점',
 '오늘은 분식',
 '이끌림마라탕',
 '탐나는피자-

In [66]:
store_name = []

for store in ygy_store_list:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

len(store_name)

167

In [67]:
store_name[:20]

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지']

- 1차 : store 이름이 풀로 들어간 경우 로 대체

In [68]:
def replace_store_names1(text):
    for name1 in store_name:
        if name1 in text:
            # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
    return text

In [69]:
replace_store_names1("1988응답하라추억의옛날도시락에서 먹은 음식은 정말 맛있었습니다.")

'<store>에서 먹은 음식은 정말 맛있었습니다.'

In [70]:
replace_store_names1("킹프레소에서 먹은 음식은 정말 맛있었습니다.") # 안바뀜

'킹프레소에서 먹은 음식은 정말 맛있었습니다.'

- 2차 : store 이름이 부분적으로 들어간 경우 store로 대체

In [71]:
def replace_store_names2(text):
    for name1 in tqdm(store_name, desc="Processing"):
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<store>", text)
    return text


In [72]:
replace_store_names2("텐텐에서 먹은 음식은 정말 맛있었습니다.") # 텐텐마라탕

Processing:   0%|          | 0/167 [00:00<?, ?it/s]

'<store>에서 먹은 음식은 정말 맛있었습니다.'

In [73]:
replace_store_names2("손수경에서 먹은 음식은 정말 맛있었습니다.") # 손수경의육회담은연어

Processing:   0%|          | 0/167 [00:00<?, ?it/s]

'<store>에서 먹은 음식은 정말 맛있었습니다.'

- 3차 : 일반명사 등은 제외하고 대체해야 함

NNG	일반 명사
NNP	고유 명사
NNB	의존 명사
NR	수사
NP	대명사
용언(V)	VV	동사
VA	형용사
VX	보조 용언
VCP	긍정 지시사(이다)
VCN	부정 지시사(아니다)
관형사	MM	관형사
부사(MA)	MAG	일반 부사
MAJ	접속 부사

In [74]:
!pip install kiwipiepy

In [75]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [76]:
tokens = kiwi.analyze(store_name)

remove_words = []

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM', 'MA', 'XR','MAG','MAJ') and len(word) == 2:
    remove_words.append(word)

remove_words

# 2글자 고유명사 제외 품사 단어들은 제외 -> 3, 4 글자 이상은 가게명

['춘리',
 '청년',
 '배달',
 '토핑',
 '조형',
 '여왕',
 '모어',
 '육회',
 '갈비',
 '피자',
 '행복',
 '명작',
 '인생',
 '고기',
 '청춘',
 '우리',
 '헬로',
 '대찌',
 '구래',
 '자성',
 '수락',
 '나인',
 '땡초',
 '피자',
 '과일',
 '삼다',
 '최고',
 '걸작',
 '직화',
 '토핑',
 '미치',
 '천년',
 '수제',
 '오늘',
 '탐나',
 '고기',
 '공주',
 '홍싸',
 '쇼부',
 '부성',
 '초밥',
 '감성',
 '짚신',
 '진지',
 '한식',
 '시민',
 '카페',
 '보도',
 '팔도',
 '도우',
 '고기',
 '고기',
 '혼밥',
 '본점',
 '상무',
 '도야',
 '냉면',
 '인생',
 '유쾌',
 '용문',
 '누보',
 '짬뽕']

In [77]:
add_remove = ['안녕','우시']

remove_words = remove_words + add_remove

In [78]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']

#위 일반명사에, 조사를 합친 경우도 제외함

In [79]:
remove_withparticles = []

for word in remove_words :
  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

- 최종 함수

In [80]:
store_name

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨족발보쌈',
 '육회',
 '갈비민족',
 '구구족',
 '피자보이시나',
 '왕빈자삼파전',
 '행복한찜닭',
 '스시사소우',
 '저팔계',
 '명작파스타',
 '올데이파스타',
 '대한냉면',
 '인생제육',
 '고기듬뿍미트박스',
 '피자스웨그',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '더팔당매운오징어',
 '우리집반찬도시락',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비',
 '대찌',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜',
 '1989마라탕',
 '미친피자',
 '천년닭강정',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은',
 '이끌림마라탕',
 '탐나는피자',
 '고기혁명',
 '라화방마라탕',
 '전금례닭볶음탕',
 '큰아들백순대',
 '공주닭발',
 '신자매김치찜',
 '짱닭치킨',
 '이태리면가게',
 '폴트버거',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부덮밥',
 '카산도',
 '부성초밥',
 '닭장수섭삼계탕',
 '깨돌이김밥',
 '울진죽변항',
 'GTS버거',
 '코브라독스',
 '마라하오',
 '땅끝수산',
 '초밥대통령',
 '올데이케밥',
 '

#### -> 적용

In [81]:

def replace_store_names(text):
    store_replace_list = []
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
            store_replace_list.append(name1)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in remove_words + remove_withparticles :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                        store_replace_list.append(part)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text, store_replace_list



In [82]:
replace_store_names('행복한 라화방마라탕에서 맛있게 식사했어요') #행복 부분이 대체안됨

('행복한 <store><store>탕에서 맛있게 식사했어요', ['마라', '라화방'])

In [83]:
replace_store_names('행복한 1988에서 맛있게 식사했어요') #행복 부분이 대체안됨

('행복한 <store>에서 맛있게 식사했어요', ['1988'])

- 대체된 부분 마스킹 칼럼으로 추가 (review아닌 reply에만!)

In [84]:
empty_list = [ [] for _ in range(len(train)) ]
train['replaced_store_names'] = empty_list

In [85]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,[비스],[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[],[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 다시 ...,0,0.685938,[],[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,[무나],[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날<menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 ...,0,0.829625,[도시락],[]
...,...,...,...,...,...,...,...
22079,29792,연탄구이 비빔밥 맛있네요 !,식사 맛있게 하셨어요 !?! 언제든지 찾아주셔도 부족함 없이 드실 수 있도록 <me...,1,0.474923,"[야채, 비빔, 고기]",[]
22080,30174,손목 아파서 아무것도 못할때 이 가게 없었으면 굶을뻔 짜글이 국물 남겨놓고 다음날 ...,고객님들의 원픽 맛집이 되기위해 최선을 다하고 있답니다! 언제든지 믿고 찾아주시면 ...,1,0.735911,[],[]
22081,14014,떡볶이 치킨 다 아주 맛있어요. 잘 먹었습니다.,고객님 덕에 너무 힘이 나네요 또 이용 해 주실거죠 ? 주말 잘 보내세요! 이용 해...,1,0.691521,[],[]
22082,27868,큼직큼직 굿굿 잘 먹었습니다,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,1,0.546858,"[초밥, 시리]",[]


In [86]:

# for i, review in tqdm(enumerate(train['review']), desc="Processing Review", ncols=100):
#     replaced_text, store_replace_list = replace_store_names(review)
#     # 'review' 칼럼에 대체된 리뷰 텍스트 저장
#     train.at[i, 'review'] = replaced_text
#     # 'replaced_store_names' 칼럼에 대체된 가게 이름 목록 저장
#     train.at[i, 'replaced_store_names'] = store_replace_list


In [87]:
for i, reply in tqdm(enumerate(train['reply']), desc="Processing Reply", ncols=100):
    text, store_replace_list = replace_store_names(reply)
    # 'reply' 칼럼에 대체된 리뷰 텍스트 저장
    train['reply'][i] = text
    # 'replaced_store_names' 칼럼에서 목록을 가져온 후, 새로운 목록을 추가
    # existing_list = train.at[i, 'replaced_store_names']
    train['replaced_store_names'][i] = store_replace_list


Processing Reply: 0it [00:00, ?it/s]

<ipython-input-87-096da5f4fc23>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = text
<ipython-input-87-096da5f4fc23>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['replaced_store_names'][i] = store_replace_list


In [88]:
# # 'replaced_store_names' 칼럼에 중복을 제거하고 고유한 값으로 변경
train['replaced_store_names'] = train['replaced_store_names'].apply(set).apply(list)

In [89]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,[비스],[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[],[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 다시 ...,0,0.685938,[],[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,[무나],[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,[도시락],"[대한, 1988응답하라추억의옛날]"
...,...,...,...,...,...,...,...
22079,29792,연탄구이 비빔밥 맛있네요 !,식사 맛있게 하셨어요 !?! 언제든지 찾아주셔도 부족함 없이 드실 수 있도록 <me...,1,0.474923,"[야채, 비빔, 고기]",[]
22080,30174,손목 아파서 아무것도 못할때 이 가게 없었으면 굶을뻔 짜글이 국물 남겨놓고 다음날 ...,고객님들의 원픽 맛집이 되기위해 최선을 다하고 있답니다! 언제든지 믿고 찾아주시면 ...,1,0.735911,[],[]
22081,14014,떡볶이 치킨 다 아주 맛있어요. 잘 먹었습니다.,고객님 덕에 너무 힘이 나네요 또 이용 해 주실거죠 ? 주말 잘 보내세요! 이용 해...,1,0.691521,[],[]
22082,27868,큼직큼직 굿굿 잘 먹었습니다,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,1,0.546858,"[초밥, 시리]",[]


In [90]:
mask = train['replaced_store_names'].str.len() > 0
train[mask]

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,[도시락],"[대한, 1988응답하라추억의옛날]"
8,92,맛이나 양은 무난해요 그런데. 배달예정시간이 무색하게 늦게도착 차라리 배달 예정시간...,<store><menu>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 ...,0,0.877273,[도시락],"[대한, 1988응답하라추억의옛날]"
9,96,"메인은 좋았는데, 아. 반찬이 다 쉬어 모두 버렸어요 한번 씹는데 촉감,맛이 이상해...",안녕하세요 <store><menu> 입니다. 우선 감사하게도 저희 매장을 찾아 주셨...,0,0.871141,"[반찬, 도시락]",[1988응답하라추억의옛날]
10,100,예정시간보다 30분이 지나서야 도착하고 음식도 다 식어서 왔네요,<store><menu>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 ...,0,0.909206,[도시락],"[대한, 1988응답하라추억의옛날]"
11,101,배달이 넘 느려서 우선 감점 배가 고픈데 그럼에도 맛은 별로 개취존중 스팸이 젤 맛...,<store><menu>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 ...,0,0.856480,[도시락],"[대한, 1988응답하라추억의옛날]"
...,...,...,...,...,...,...,...
22068,12123,맛있게 잘 먹었습니다 김치전도 잘 먹었습니다,감사합니다 맛있게드셔주셧다니 마음<menu> 편안합니다 언제나 맛으로 보답드리는 <...,1,0.521267,[가지],[수락]
22072,1523,고돼지 언제나 맛있습니다 잘먹었습니당,고<menu> <store>점입니다 별점만점 그리고 따뜻한 리뷰 작성해주셔서 감사합...,1,0.593685,"[고기, 돼지]",[송파]
22075,36054,잘먹겠습니다 오늘도.,<store> 객사점 입니다 멋진 리뷰 너무 감사드립니다 항상 변치않고 좋은 음식으...,1,0.675867,[비스],[백소정]
22077,23045,엄청 맛있는 편은 아니지만 꽤 먹을만합니다! 리뷰이벤트로 주신 마늘돈가스.흰 밥이랑...,소중한 리뷰 감사합니다 다음에는 <menu><store> 드셔보세용. 청주에는 저희...,1,0.416850,[마늘],[돈까스]


### 지점명 처리

- 호점 삭제

In [91]:
# 정규 표현식을 사용하여 "~호점" 패턴을 찾아서 삭제하는 함수
def remove_hojum(text):
    return re.sub(r'\w+\s*호점','', text)

# train 데이터프레임의 'reply' 열에 적용하여 "~호점"을 삭제
train['reply'] = train['reply'].apply(remove_hojum)
train['review'] = train['review'].apply(remove_hojum)


In [92]:
# try:
#     restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv", encoding='cp949')
# except pd.errors.ParserError:
#     pass

In [93]:
restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')

<ipython-input-93-afda4487c689>:1: DtypeWarning: Columns (11,27,39,41,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')


In [94]:
# try:
#     restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv", encoding='cp949')
# except pd.errors.ParserError:
#     pass  # 오류 행을 무시합니다.

In [95]:
restaurant_list

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장생산직직원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지,Unnamed: 47
0,1,일반음식점,07_24_04_P,3050000,3050000-101-2015-00184,2015-06-17,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,N,26.40,NaN,NaN,NaN,NaN
1,2,일반음식점,07_24_04_P,3130000,3130000-101-1992-03288,1992-01-24,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,N,140.75,NaN,NaN,NaN,NaN
2,3,일반음식점,07_24_04_P,3130000,3130000-101-1992-03063,1992-01-22,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,N,83.51,NaN,NaN,NaN,NaN
3,4,일반음식점,07_24_04_P,3130000,3130000-101-2000-08298,2000-03-15,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,N,117.70,NaN,NaN,NaN,NaN
4,5,일반음식점,07_24_04_P,3830000,3850000-101-2013-00086,2013-04-11,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,N,20.38,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086169,2086170,일반음식점,07_24_04_P,3600000,3600000-101-1995-00101,1995-09-01,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,68.50,NaN,NaN,NaN,NaN
2086170,2086171,일반음식점,07_24_04_P,3610000,3610000-101-2012-00009,2012-01-19,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,85.00,NaN,NaN,NaN,NaN
2086171,2086172,일반음식점,07_24_04_P,3610000,3610000-101-2014-00098,2014-06-12,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,N,118.98,NaN,NaN,NaN,NaN
2086172,2086173,일반음식점,07_24_04_P,3600000,3600000-101-2014-00060,2014-03-21,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,N,26.40,NaN,NaN,NaN,NaN


In [96]:
# restaurant_list['소재지전체주소']

In [97]:
# 가게명에서 지점명 추출하기

store_etc_name = []

for store in ygy_store_list:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

ygy_region_name = []

for region in store_etc_name:
  if region[-1] =='점':
    ygy_region_name.append(region)

In [98]:
address_parts=[]

for address in restaurant_list['소재지전체주소'].astype(str):
    if address.startswith('서울'or '부산' or '청주'):
        parts = address.split(' ')
        for _ in range(0, min(4, len(parts))):  # 최대 5개 또는 주소 부분 개수만큼 반복
          part = parts[_]
          address_parts.append(part)

address_parts = set(address_parts)
address_parts = list(address_parts)

address_parts = [name for name in address_parts if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움
address_parts.remove('')

In [99]:
address_parts2 = [] # 추출된 지역명을 지점명으로 변경

for address in address_parts :
  if len(address) < 3  :
    address = address + '점'
    address_parts2.append(address)
  else :
    address = address[:-1] + '점'
    address_parts2.append(address)

In [100]:
ygy_store_list

['1988응답하라추억의옛날도시락-방이점',
 '워커스커피 로스터스',
 '떡깨비-가락점',
 '당치땡-헬리오시티점',
 '우시장국밥-석촌점',
 '고돼지-송파점',
 '마라의신마라탕&마라샹궈',
 '춘리마라탕-송파점',
 '손수경의육회담은연어-가락점',
 '무국적식탁-송파점',
 '청년치킨-삼전점',
 '송파만다린',
 '배달의쌀국수-송파점',
 '닭강점-헬리오시티점',
 '토핑폭탄김치찜&김치찌개-송파점',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82-송파점',
 '더티베지',
 '잭아저씨족발보쌈-본점',
 '육회&연어-미아본점',
 '갈비민족-본점',
 '구구족-성신여대역점',
 '피자보이시나-대학로점',
 '왕빈자삼파전-미아본점',
 '행복한찜닭-성북점',
 '스시사소우',
 '저팔계&중국&마라탕&덮밥&요리&전문점',
 '명작파스타-성북점',
 '올데이파스타-동대문구점',
 '대한냉면-성북점',
 '인생제육',
 '고기듬뿍미트박스-성북점',
 '피자스웨그-성북점',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다-북서울꿈의숲점',
 '더팔당매운오징어&닭발 김치찜',
 '우리집반찬도시락-혜화막내딸지점',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비&마약볶음밥',
 '대찌 대파생고기김치찌개&정육왕국물두루치기',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜&김치찌개-부천점',
 '1989마라탕-부천점',
 '미친피자-부평점',
 '천년닭강정-부천부평점',
 '수상한삼겹살-부천점',
 '수제죽전문점-본가진죽-본점',
 '오늘은 분식',
 '이끌림마라탕',
 '탐나는피자-

In [101]:
# 가게명에서 지점명 추출하기

store_etc_name = []

for store in ygy_store_list:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

ygy_region_name = []

for region in store_etc_name:
  if region[-1] =='점':
    ygy_region_name.append(region)

In [102]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,[비스],[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[],[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 다시 ...,0,0.685938,[],[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,[무나],[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,[도시락],"[대한, 1988응답하라추억의옛날]"
...,...,...,...,...,...,...,...
22079,29792,연탄구이 비빔밥 맛있네요 !,식사 맛있게 하셨어요 !?! 언제든지 찾아주셔도 부족함 없이 드실 수 있도록 <me...,1,0.474923,"[야채, 비빔, 고기]",[]
22080,30174,손목 아파서 아무것도 못할때 이 가게 없었으면 굶을뻔 짜글이 국물 남겨놓고 다음날 ...,고객님들의 원픽 맛집이 되기위해 최선을 다하고 있답니다! 언제든지 믿고 찾아주시면 ...,1,0.735911,[],[]
22081,14014,떡볶이 치킨 다 아주 맛있어요. 잘 먹었습니다.,고객님 덕에 너무 힘이 나네요 또 이용 해 주실거죠 ? 주말 잘 보내세요! 이용 해...,1,0.691521,[],[]
22082,27868,큼직큼직 굿굿 잘 먹었습니다,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,1,0.546858,"[초밥, 시리]",[]


In [103]:
region_name = ygy_region_name + address_parts2

In [104]:
region_name = set(region_name)
region_name = list(region_name)

In [105]:
region_name


['종로점',
 '청량리점',
 '무지개쇼점',
 '필운점',
 '주교점',
 '효제점',
 '전주1호점',
 '신도림점',
 '내곡점',
 '서대전점',
 '북서울꿈의숲점',
 '교북점',
 '잠원점',
 '연희점',
 '광장점',
 '중구점',
 '다동점',
 '은평점',
 '신설점',
 '성북점',
 '평동점',
 '상당구점',
 '신수점',
 '강서점',
 '옥수점',
 '충신점',
 '신내점',
 '방화점',
 '하월곡점',
 '양천점',
 '예지점',
 '석관점',
 '구수점',
 '도선점',
 '양화점',
 '삼청점',
 '청담점',
 '장안점',
 '과해점',
 '성내점',
 '서초점',
 '암사점',
 '안국점',
 '공덕점',
 '권농점',
 '상봉점',
 '거여점',
 '갈월점',
 '통인점',
 '갈현점',
 '가락점',
 '이화점',
 '풍립빌점',
 '수색점',
 '혜화막내딸지점',
 '장교점',
 '후암점',
 '동숭점',
 '우이점',
 '관수점',
 '부암점',
 '소격점',
 '창동점',
 '조대점',
 '동작점',
 '통의점',
 '한남점',
 '개봉점',
 '용강점',
 '강북점',
 '남가좌점',
 '광주광천점',
 '율현점',
 '대신점',
 '누상점',
 '외발산점',
 '아현점',
 '홍지점',
 '정동점',
 '용인수지점',
 '시흥점',
 '등촌점',
 '도렴점',
 '부천부평점',
 '돈암점',
 '누하점',
 '강동점',
 '마천점',
 '분당점',
 '판교점',
 '대덕테크노벨리점',
 '냉천점',
 '중동점',
 '대현점',
 '관철점',
 '당인점',
 '수하점',
 '방산점',
 '신중동점',
 '논현점',
 '용암점',
 '하중점',
 '오금점',
 '대전역점',
 '회기점',
 '대조점',
 '화동점',
 '망원점',
 '반포점',
 '명일점',
 '청암점',
 '서빙고점',
 '대학로점',
 '신정점',
 '장위점',
 '서현본점',
 '연건점',
 '신림점',
 '옥인점',
 '목동충촌점',
 '

In [106]:
def replace_region_names(text):
    region_replace_list = []
    for name1 in region_name:
        if name1 in text:
            # 지점 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<region>", text)
            region_replace_list.append(name1)
        # else :
        #   for i in range(len(name1), 1, -1):
        #       part = name1[:i]  # 지점 이름에서 처음부터 i 글자까지의 부분
        #       if part in text:
        #           # 지점 이름의 일부가 텍스트에 포함되어 있다면 대체
        #           pattern = re.compile(re.escape(part), re.IGNORECASE)
        #           text = pattern.sub("<region>", text)
    return text, region_replace_list


In [107]:
replace_region_names('오랜만에 헬리오시티점에서 맛있게 식사했어요')

('오랜만에 <region>에서 맛있게 식사했어요', ['헬리오시티점'])

In [108]:
empty_list = [ [] for _ in range(len(train)) ]
train['replaced_region_names'] = empty_list

NameError: ignored

In [110]:


for i, reply in tqdm(enumerate(train['reply']), desc="Processing Reply", ncols=100):
    text, region_replace_list = replace_region_names(reply)
    # 'reply' 칼럼에 대체된 리뷰 텍스트 저장
    train['reply'][i] = text
    # 'replaced_store_names' 칼럼에서 목록을 가져온 후, 새로운 목록을 추가
    # existing_list = train.at[i, 'replaced_store_names']
    train['replaced_region_names'][i] = region_replace_list

Processing Reply: 0it [00:00, ?it/s]

<ipython-input-110-f8e691d7ef9f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = text
<ipython-input-110-f8e691d7ef9f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['replaced_region_names'][i] = region_replace_list


In [111]:
mask = train['reply'].str.contains('<region>')
train[mask]

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names,replaced_region_names
43,593,잠실점에 처음 시켜먹었던게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 들어와보니...,요기요가 주<menu>플에 렉이 자주 있어서. 여러<menu>로 혼돈이 있어 정말 ...,0,0.995390,"[가지, 문어]",[],[잠실점]
72,925,맛은 준수 양은 베리굿 다만 떡볶이가 사실상 캡사이신 범벅 얼마나 맛있게 맵냐가 중...,고객님 <store> <region>입니다. <menu>에 조금 아쉬움이 있으신군요...,0,0.741032,[떡볶이],[당치땡],[헬리오시티점]
75,963,배달이 너무 늦네요 시간도 오버되고 치킨은 거의 다 식었네요,고객님께 즐거움을 드려야 하는데 그러 지 못한점 <menu> 드립니다. 배달 대행업...,0,0.903930,[사과],[당치땡],[헬리오시티점]
76,975,오늘 월드컵 경기 때문에 배달이 많이 밀려 늦은것은 이해합니다만 그렇다하더라 치킨의...,안녕하세요 당신은 지금 <menu>이 땡긴다 <region> 입니다. 이번 월드컵 ...,0,0.976034,[치킨],[당치땡],[헬리오시티점]
77,977,8시40분에 주문했는데 11시10분에 받음 2시간 40분걸림 전화도 안됨 아무리 월...,안녕하세요 당신은 지금 <menu>이 땡긴다 <region> 입니다. 이번 월드컵 ...,0,0.991011,[치킨],[당치땡],[헬리오시티점]
...,...,...,...,...,...,...,...,...
22057,23998,푸짐하게 맛잇게 잘먹었습니다,안녕하세요 <store><menu>끼 청주점입니다 맛있게 드셨다니 열심히 구운 보람...,1,0.567574,"[타코야, 중국, 수제]",[모모],[전문점]
22059,24017,잘먹었습니다 ! 너무 맛있어옹,안녕하세요 <store><menu>끼 청주점입니다 맛있게 드셨다니 열심히 구운 보람...,1,0.710885,"[타코야, 중국, 수제]",[모모],[전문점]
22074,33264,빠르고 잘 먹었습니다,저희 도야<menu> <region>을 이용해주셔서 감사합니다 별 다섯개 리뷰 감사...,1,0.565116,[짬뽕],[],[계림점]
22075,36054,잘먹겠습니다 오늘도.,<store> <region> 입니다 멋진 리뷰 너무 감사드립니다 항상 변치않고 좋...,1,0.675867,[비스],[백소정],[객사점]


In [112]:

train['replaced_store_names'] = train['replaced_store_names'].apply(set).apply(list)

In [113]:
# train.to_csv(f'train_0920_22170.csv',index=False)

train.to_csv(f'train_0921_22170.csv',index=False)

# 모델 학습 및 추론 (소현님 파일)
- 이소현_서비스2_모델링2.ipynb

In [114]:
!pip install transformers


In [115]:
model_name = "skt/kogpt2-base-v2"

In [116]:

# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [117]:
model = AutoModelForCausalLM.from_pretrained(model_name)


In [118]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

In [119]:
train.columns

Index(['index', 'review', 'reply', 'sentiment', 'score', 'replaced_menu_names',
       'replaced_store_names', 'replaced_region_names'],
      dtype='object')

In [120]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        sentiment = item["sentiment"]
        store = item["store"]
        region = item["region"]
        menu = item["menu"]
        review = item["review"]
        # review = item["keyword"]
        reply = item["reply"]

        return {
            "sentiment": sentiment,
            "store": store,
            "region": region,
            "menu": menu,
            "review": review,
            "reply": reply
        }

In [121]:
def collate_fn(batch):
    sentiment = torch.tensor([item["sentiment"] for item in batch])
    texts = [
        f"<w>{item['store']}</s><b>{item['region']}</s><f>{item['menu']}</s><q>{item['review']}</s><a>{item['reply']}</s>"
        for item in batch
    ]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x, "sentiment": sentiment}

In [122]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names,replaced_region_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,[비스],[],[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[],[],[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 다시 ...,0,0.685938,[],[],[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,[무나],[],[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,[도시락],"[대한, 1988응답하라추억의옛날]",[]
...,...,...,...,...,...,...,...,...
22079,29792,연탄구이 비빔밥 맛있네요 !,식사 맛있게 하셨어요 !?! 언제든지 찾아주셔도 부족함 없이 드실 수 있도록 <me...,1,0.474923,"[야채, 비빔, 고기]",[],[]
22080,30174,손목 아파서 아무것도 못할때 이 가게 없었으면 굶을뻔 짜글이 국물 남겨놓고 다음날 ...,고객님들의 원픽 맛집이 되기위해 최선을 다하고 있답니다! 언제든지 믿고 찾아주시면 ...,1,0.735911,[],[],[]
22081,14014,떡볶이 치킨 다 아주 맛있어요. 잘 먹었습니다.,고객님 덕에 너무 힘이 나네요 또 이용 해 주실거죠 ? 주말 잘 보내세요! 이용 해...,1,0.691521,[],[],[]
22082,27868,큼직큼직 굿굿 잘 먹었습니다,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,1,0.546858,"[초밥, 시리]",[],[]


In [123]:
train['replaced_menu_names'] = train['replaced_menu_names'].apply(lambda x: ', '.join(x))
train['replaced_store_names'] = train['replaced_store_names'].apply(lambda x: ', '.join(x))
train['replaced_region_names'] = train['replaced_region_names'].apply(lambda x: ', '.join(x))

In [124]:
train.columns = ['index','review','reply','sentiment','score', 'menu','store','region']

In [125]:
dt = ChatDataset(train)
dt[0]

{'sentiment': 0,
 'store': '',
 'region': '',
 'menu': '비스',
 'review': '기름 있는 부분이 생각보다 많아요! 참고하세요 ',
 'reply': '소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있도록 노력 하겠습니다! 고객 관점에서 불편함이 없는지, 아쉬운 점이 없는지 끊임없이 고민하고, 개선해 나아가 앞으로도 기대에 실망 시키지 않고, 더 맛있는 맛과 서<menu>를 제공하는 것을 약속하겠습니다 다시 주문 주셔도 방심하지 않고 지금과 같은 자세로 대기하고 있을게요 그때까지 건강 유념하시길 바라요!'}

In [126]:
# df = train[['review','reply','sentiment',

In [127]:
dt = ChatDataset(train)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   461,   405,     1,  9724,   440,   405,     1,  9724,   444,
            405, 10425,     1,  9724,   455,   405, 19824,  9080, 13748,  9658,
           9518, 12268,  8084,   376, 11522,  8702,  7801,  8084,   739,     1,
           9724,   439,   405,  7824, 12617, 11955, 18525,  9046,  7810,  7788,
          15940, 37194,  9443, 11939, 12904, 12634,  9947,  9267, 12003,  9337,
           9025, 10517, 10805, 14651, 16913,  7182,   376, 12298, 17662, 19092,
          14316,  9712, 10262, 30939,  8094, 12425,  9712,  8263, 16591, 31279,
           9307, 12003,  8711, 12717, 12634,  7235,  9032,  9875, 40974, 42401,
           8263, 14439,  9267, 48541, 23600,  9071,   403,   451,  9714,   459,
          22270, 15662,  9290, 13274, 26616, 16913,  7182,  9427, 16812,  9046,
           7810,  7235,  9127,  7894,  9328,  9801,  9782,  6903,  9239, 33204,
          13346,  9038,  9846,  6866,  8084, 28359, 11342,  9055,  7111, 28944,
           6962, 1315

In [128]:
model(**batch["x"]).logits.shape

torch.Size([2, 141, 51200])

In [129]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"매개변수 {name}는 requires_grad가 True입니다.")
    else:
        print(f"매개변수 {name}는 requires_grad가 False입니다.")

매개변수 transformer.wte.weight는 requires_grad가 True입니다.
매개변수 transformer.wpe.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_1.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_1.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_attn.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_attn.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_proj.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_proj.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_2.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_2.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_fc.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_fc.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_proj.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_proj.bias는 requires_grad가 True입니다.
매개변수 transformer.h.1.ln_1.weight는 requires_grad가 True입니다.
매개변수 transformer.h.1.ln_1.bias는 requires_grad가 True입니다.
매개변수 transformer.h.1.attn.c_attn.weight는 require

In [130]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [131]:
for name,param in model.named_parameters():
    if name.startswith("transformer.h.11"):
        break
    param.requires_grad = False

for name, param in model.named_parameters():
    print(name,param.requires_grad)

transformer.wte.weight False
transformer.wpe.weight False
transformer.h.0.ln_1.weight False
transformer.h.0.ln_1.bias False
transformer.h.0.attn.c_attn.weight False
transformer.h.0.attn.c_attn.bias False
transformer.h.0.attn.c_proj.weight False
transformer.h.0.attn.c_proj.bias False
transformer.h.0.ln_2.weight False
transformer.h.0.ln_2.bias False
transformer.h.0.mlp.c_fc.weight False
transformer.h.0.mlp.c_fc.bias False
transformer.h.0.mlp.c_proj.weight False
transformer.h.0.mlp.c_proj.bias False
transformer.h.1.ln_1.weight False
transformer.h.1.ln_1.bias False
transformer.h.1.attn.c_attn.weight False
transformer.h.1.attn.c_attn.bias False
transformer.h.1.attn.c_proj.weight False
transformer.h.1.attn.c_proj.bias False
transformer.h.1.ln_2.weight False
transformer.h.1.ln_2.bias False
transformer.h.1.mlp.c_fc.weight False
transformer.h.1.mlp.c_fc.bias False
transformer.h.1.mlp.c_proj.weight False
transformer.h.1.mlp.c_proj.bias False
transformer.h.2.ln_1.weight False
transformer.h.2.ln_1

In [132]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):

        x = batch["x"].to(device)
        sentiment = batch["sentiment"].to(device)

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수
        sentiment_pred = pred[:, -1, :]

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]

        loss = loss_fn(pred, tgt)

        sentiment_loss = loss_fn(sentiment_pred, sentiment)

        optimizer.zero_grad()
        total_loss = loss + sentiment_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [133]:
shuffled_train = train.sample(frac=1, random_state=42)  # random_state는 무작위 섞기를 위한 시드값입니다.
df = shuffled_train[:16000]
df

,index,review,reply,sentiment,score,menu,store,region
16316,21525,맛있게 먹었습니다,이쁜 리뷰 감사합니다 가성비 1등 <store> 충북대점을 찾아주셔서 감사합니다 앞...,1,0.486165,알밥,알촌,
7744,6017,주문한 음식이 아예 다른 음식으로 바뀐 것 같아요. 정말 실망이었어요,"틀린 음식으로 전달된 것 같아 죄송해요. 정말 실망스러웠을 텐데, 다시는 그런 일이...",0,0.938211,,,
221,4418,배달이 좀 느리네요. 맛은 보통이엇습니다 .,<store><menu><menu><menu>을 믿고 주문해주셔서 감사합니다 다음 ...,0,0.887333,"보쌈, 도시락, 비스, 족발",조형훈,
7858,6131,너무 기대한 만큼은 아니었어요. 피자도 맛이 별로였고 토핑도 별로였어요,<menu>가 늦게 도착하고 맛도 별로였다니 저도 실망스럽네요. 더 향상된 서<me...,0,0.915643,"비스, 피자",,
13925,2217,맛있게 잘먹었습니다 급하게 먹느라 사진을 못찍었네요 다음에는 혼술세트로 시켜먹으려구...,맛있게 드셔주셔서 감사드립니다 늘 믿고 드실 수 있게 항상 신선한 재료로 정성을 듬...,1,0.694568,,,
...,...,...,...,...,...,...,...,...
2135,395,"배달이 너무 느려서 기다리는 동안 배가 고팠어요. 음식도 따뜻하지 않았고, 별로 맛...",안녕하세요. 배달이 느려서 기다리는 동안 힘드셨을 것 같네요. 더욱 빠르고 신선한 ...,0,0.938337,,,
18563,2157,먹기 전부터 존맛에 기운이 느껴짐 춘리는 역시춘리 믿고 먹어요 맛있게 자알 묵었습니다,맛있게 드셔주셔서 감사합니다 요청사항은 조리전 늘 꼼꼼하게 체크하고 있으니 필요하신...,1,0.744525,,대한,
10861,9142,맛이 별로였어요. 약간 썩은 맛이 났어요. 못 먹겠더라고요,맛이 나지 않고 약간 썩은 맛이 난 것 같아 죄송합니다. 더 맛있는 음식으로 보답하...,0,0.920898,,,
20743,10344,맛있고 괜찮았는데 소스도 조금 싸주시면 더 좋았을것 같아요,고객님 덕분에 큰 힘이 나네요 불끈 불끈 저희는 찍어먹는 <menu>는 따로 없고 ...,1,0.464085,"곱창, 소스",삽교원조두리,


In [135]:
reset_seeds(SEED)
batch_size = 2
epochs = 10

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

## 동결 코드 ###
for name,param in model.named_parameters():
    if name.startswith("transformer.h.4"):
        break
    param.requires_grad = False

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(df)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")


  0%|          | 0/8000 [00:00<?, ?it/s]

1번째 epoch: 1.8746


  0%|          | 0/8000 [00:00<?, ?it/s]

2번째 epoch: 1.4831


  0%|          | 0/8000 [00:00<?, ?it/s]

...

8번째 epoch: 0.9601


  0%|          | 0/8000 [00:00<?, ?it/s]

9번째 epoch: 0.8982


  0%|          | 0/8000 [00:00<?, ?it/s]

10번째 epoch: 0.8404


In [136]:
model.save_pretrained(f"{DATA_PATH}gpt_22000_0921")

# 답변 생성

In [137]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}gpt_22000_0921").to(device)

In [138]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 951.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ac34e3392ff3550aba9f938440f15f86c498ebc844011551dd9611e6d7be78d3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [139]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text):
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    similarity_score = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0] # cosine
    # similarity_score = pairwise_distances([input_embedding], [generated_embedding], metric='euclidean')[0][0] # euclidean
    # similarity_score = pairwise_distances([input_embedding], [generated_embedding], metric='manhattan')[0][0] # manhattan

    return similarity_score

In [140]:
# 옵션
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=512, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=50,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [141]:
# def chatbot_with_evaluation(model, tokenizer, max_len, device, store, menu, region, num_samples=3) :
#     model.eval()
#     while True:
#         review = input("user > ").strip()
#         if review == "quit":
#             break

#         review = review.replace('<itore>',store)
#         review = review.replace('<menu>',menu)
#         review = review.replace('<region>',region)


#         text = "<q>" + review + "</s><a>"
#         x = tokenizer.encode(text, return_tensors="pt").to(device)
#         q_len = len(text) + 1

#         best_generated_text = None
#         best_similarity_score = -1.0
#         generated_texts = []  # 답변 후보군을 담을 리스트
#         for i in range(num_samples):
#             result_ids = model.generate(x,
#                                         max_length=max_len,
#                                         repetition_penalty=2.0,
#                                         use_cache=True,
#                                         do_sample=False,
#                                         temperature=0.9,
#                                         # top_k=50,
#                                         top_p = 0.95,
#                                         num_return_sequences=1
#                                         )
#             generated_text = tokenizer.decode(result_ids[0])
#             generated_text = generated_text[q_len:-4]
#             # generated_text = paraphrase_text(generated_text)

#             generated_text = generated_text.replace('<itore>',store)
#             generated_text = generated_text.replace('<menu>',menu)
#             generated_text = generated_text.replace('<region>',region)


#             similarity_score = evaluate_similarity(text, generated_text)
#             generated_texts.append((similarity_score,generated_text))
#             print(generated_texts[i])

#             if similarity_score > best_similarity_score:
#                 best_similarity_score = similarity_score
#                 best_generated_text = generated_text

#         print()
#         print("Best_reply >", best_generated_text)
#         print("Best_similarity_score:", best_similarity_score)
#         print("--------------------------------------------------------------------------------------------")


In [142]:
chatbot_with_evaluation(model,tokenizer,128,device, '최가네','닭갈비','천호점')

user > 맛있어요


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


(0.4467661380767822, '고객님께 사랑받도록 하겠습니다. 믿고 주문해주실거<unk>. 생각나시면 이용해주세요. 즐거운 하루 되시고 날씨가 추워졌으니 감기조심하셔용! 건강 조심하시고 행복한 나날되십시오. 감사합니다. 새해 복 많이 받으시기 바랍니다. 또 만나뵈었으면 좋겠습니다 2011708-1번가주서 경계할까봐 확인드려도 좋을것 같네욤 소중한 리뷰 너무너무감사드립니당 오늘하루 행복만 가득 하시길바랄게여 안녕 하십')


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


(0.4467661380767822, '고객님께 사랑받도록 하겠습니다. 믿고 주문해주실거<unk>. 생각나시면 이용해주세요. 즐거운 하루 되시고 날씨가 추워졌으니 감기조심하셔용! 건강 조심하시고 행복한 나날되십시오. 감사합니다. 새해 복 많이 받으시기 바랍니다. 또 만나뵈었으면 좋겠습니다 2011708-1번가주서 경계할까봐 확인드려도 좋을것 같네욤 소중한 리뷰 너무너무감사드립니당 오늘하루 행복만 가득 하시길바랄게여 안녕 하십')


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


(0.4467661380767822, '고객님께 사랑받도록 하겠습니다. 믿고 주문해주실거<unk>. 생각나시면 이용해주세요. 즐거운 하루 되시고 날씨가 추워졌으니 감기조심하셔용! 건강 조심하시고 행복한 나날되십시오. 감사합니다. 새해 복 많이 받으시기 바랍니다. 또 만나뵈었으면 좋겠습니다 2011708-1번가주서 경계할까봐 확인드려도 좋을것 같네욤 소중한 리뷰 너무너무감사드립니당 오늘하루 행복만 가득 하시길바랄게여 안녕 하십')

Best_reply > 고객님께 사랑받도록 하겠습니다. 믿고 주문해주실거<unk>. 생각나시면 이용해주세요. 즐거운 하루 되시고 날씨가 추워졌으니 감기조심하셔용! 건강 조심하시고 행복한 나날되십시오. 감사합니다. 새해 복 많이 받으시기 바랍니다. 또 만나뵈었으면 좋겠습니다 2011708-1번가주서 경계할까봐 확인드려도 좋을것 같네욤 소중한 리뷰 너무너무감사드립니당 오늘하루 행복만 가득 하시길바랄게여 안녕 하십
Best_similarity_score: 0.4467661380767822
--------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [162]:
max_len = 512

def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=3) :
    model.eval()
    while True:
        review = input("user > ").strip()
        if review == "quit":
            break

        review = review.replace('<itore>','')
        review = review.replace('<menu>','')
        review = review.replace('<region>','')

        text = "<q>" + review + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=3.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=1.05,
                                        top_k=60,
                                        num_beams=2,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=5,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            generated_text = paraphrase_text(generated_text)

            generated_text = generated_text.replace('<itore>','')
            generated_text = generated_text.replace('<menu>','')
            generated_text = generated_text.replace('<region>','')


            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)

        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [163]:

# 0908_gpt_freeze
chatbot_with_evaluation(loaded_model,tokenizer,512,device)

KeyboardInterrupt: ignored

#

# 답변생성 + 키워드

In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 45.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=b236ae56f16edf05fca7c90c7007789d6c2fe2edb5e1e518d49f8c91a9394ff1
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [ ]:
!pip install keybert


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.5 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=8c9bf22a7ec600bf661ddf856f1fa20fbab686bac817586741933c8727a37668
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b7b9bb1b377097729e2a2dc5cc45fd918a1ddc971701b13b291a97ec7db6bfa7
 

In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [ ]:

kiwi_reviews = []

for review in tqdm(train_ft['review']):
    results = []  # 결과 리스트를 각 리뷰마다 초기화
    result = kiwi.analyze(review)
    for token, pos, _, _ in result[0][0]:
        if pos in ['NNG', 'VA', 'VV', 'XR']:  # 일반명사, 형용사, 동사, 어근
            results.append(token)
    kiwi_reviews.append(results)


  0%|          | 0/45124 [00:00<?, ?it/s]

In [ ]:
train_ft['kiwi_reviews'] = kiwi_reviews


In [ ]:
# 우선은 이전에 뽑아놨던 키워드 활용


# import re

# total_keybert = []

# for i in train_ft['keybert_reviews']:
#     korean_words = re.findall(r'[가-힣]+', i)  # ' ' 사이의 한글 단어 추출
#     total_keybert.extend(korean_words)



In [ ]:
# from collections import Counter

# word_counts = Counter(total_keybert)

# sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# for word, count in sorted_word_counts[:200]:
#     print(f"{word}: {count}")


In [ ]:

#200위 내 키워드들 중에 맛, 양, 가격, 서비스, 배달 관련 키워드 추출
taste_keyword = ['맛','맛있','맛나','맛집','매콤하','느끼','신선','냄새','입맛','신선하','건강','매콤','싱싱하','잡내','매운맛','달달하']
quantity_keyword = ['양','푸짐','푸짐하','양도','넉넉','다양','든든','두툼','부족','배부르','든든하'] #'양'추가함
price_keyword = ['가성비','가격','비싸','저렴하']
service_keyword = ['서비스','깔끔','이벤트','친절']
delivery_keyword = ['배달','빠르','배송','도착']


# #100위 내 키워드들 중 맛, 양, 가격, 서비스, 배달 관련 키워드 추출
# taste_keyword = ['맛있','맛나','맛집','매콤하','신선하','냄새']
# quantity_keyword = ['양','푸짐','푸짐하','넉넉','다양','두툼','배부르'] #'양'추가함
# price_keyword = ['가성비','가격']
# service_keyword = ['서비스','깔끔','이벤트','친절']
# delivery_keyword = ['배달','빠르','배송']

In [ ]:
train['category'] = None

In [ ]:
def get_category(words):
    found_categories = set()  # 중복을 제외한 카테고리를 저장할 집합

    for kiwi in words:
        if kiwi in taste_keyword:
            found_categories.add('맛')
        elif kiwi in quantity_keyword:
            found_categories.add('양')
        elif kiwi in price_keyword:
            found_categories.add('가격')
        elif kiwi in service_keyword:
            found_categories.add('서비스')
        elif kiwi in delivery_keyword:
            found_categories.add('배달')

    # 집합을 리스트로 변환
    unique_categories = list(found_categories)

    return unique_categories

In [ ]:
train_ft['category'] = train_ft['kiwi_reviews'].apply(lambda words: get_category(words))


In [ ]:
train_ft

,review,reply,sentiment,score,kiwi_reviews,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494,"[맛있, 먹, 묵사발, 시원, 맛있]",[맛]
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,"[기름, 있, 부분, 생각, 많, 참, 하]",[]
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408,"[양, 푸짐, 맛있, 청국장, 냄새, 나, 콩, 들, 좋]","[양, 맛]"
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793,"[냉면, 시키, 미니, 냉면, 합치, 양, 적, 보이, 김치, 정식, 양, 적, 보...",[양]
5,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870,"[맛나, 묵]",[맛]
...,...,...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769,"[맛, 음식, 촉촉, 식, 먹, 그렇, 다음, 시키, 같]",[맛]
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723,"[맛, 없, 배달, 늦, 실망]","[맛, 배달]"
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588,"[최악, 맛, 없, 배달, 걸리]","[맛, 배달]"
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님들께 더 맛있는 음식을 제공하기 위해 항상 노력하고 있습니다. 맛과 냄새에 대...,0,0.933854,"[음식, 시큼, 입, 맞]",[]


In [ ]:

train_ft = train_ft[train_ft['category'].apply(lambda x: x != '[]' and len(x) > 0)]
train_ft.shape
train_ft

,review,reply,sentiment,score,kiwi_reviews,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494,"[맛있, 먹, 묵사발, 시원, 맛있]",[맛]
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408,"[양, 푸짐, 맛있, 청국장, 냄새, 나, 콩, 들, 좋]","[양, 맛]"
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793,"[냉면, 시키, 미니, 냉면, 합치, 양, 적, 보이, 김치, 정식, 양, 적, 보...",[양]
5,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870,"[맛나, 묵]",[맛]
6,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",1,0.778149,"[맛, 양, 만족, 배달, 보내, 만족]","[양, 맛, 배달]"
...,...,...,...,...,...,...
9316,항상 맛있게 먹었는데 이번에는 음식이 식어서 별로였어요. 아쉬워요,고객님의 의견을 반영하여 다양한 맛과 양을 제공하도록 노력하겠습니다.,0,0.811507,"[맛있, 먹, 이번, 음식, 식]",[맛]
9317,언제나 맛있게 먹었는데 이번에는 음식이 지루하고 별로였어요. 아쉬워요,"닭도리탕의 맛이 매우 좋지 않고, 김치가 물렸다는 점 죄송합니다. 앞으로 더 맛있는...",0,0.827452,"[맛있, 먹, 이번, 음식, 지루하]",[맛]
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769,"[맛, 음식, 촉촉, 식, 먹, 그렇, 다음, 시키, 같]",[맛]
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723,"[맛, 없, 배달, 늦, 실망]","[맛, 배달]"


In [ ]:
train_ft[['review','sentiment','score','category']]


,review,sentiment,score,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,1,0.566494,[맛]
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,1,0.608408,"[양, 맛]"
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,1,0.381793,[양]
5,맛나게 잘 묵었습니다,1,0.549870,[맛]
6,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,1,0.778149,"[양, 맛, 배달]"
...,...,...,...,...
9316,항상 맛있게 먹었는데 이번에는 음식이 식어서 별로였어요. 아쉬워요,0,0.811507,[맛]
9317,언제나 맛있게 먹었는데 이번에는 음식이 지루하고 별로였어요. 아쉬워요,0,0.827452,[맛]
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",0,0.942769,[맛]
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",0,0.912723,"[맛, 배달]"


In [181]:


max_len = 512

def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=3) :
    model.eval()
    while True:
        review = input("user > ").strip()
        if review == "quit":
            break

        review = review.replace('<itore>','')
        review = review.replace('<store>','')
        review = review.replace('<menu>','')
        review = review.replace('<region>','')
        review = review.replace('<unk>','')

        text = "<q>" + review + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=5.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.95,
                                        top_k=30,
                                        num_beams=2,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=2,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)

            generated_text = generated_text.replace('<itore>','')
            generated_text = generated_text.replace('<store>','')
            generated_text = generated_text.replace('<menu>','')
            generated_text = generated_text.replace('<region>','')
            generated_text = generated_text.replace('<unk>','')



            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)

        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [182]:



loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}gpt_22000_0921").to(device)
# 0908_gpt_freeze
chatbot_with_evaluation(loaded_model,tokenizer,512,device)

user > 치킨이 너무 맛있어요
(0.31089699268341064, '고객님께서 이 땡기실 때마다 저희가 생각나도록 더욱 열심히 하겠습니다아 주문 정말 감사드리고 다음에 또 봐요 ')
(0.31089699268341064, '고객님께서 이 땡기실 때마다 저희가 생각나도록 더욱 열심히 하겠습니다아 주문 정말 감사드리고 다음에 또 봐요 ')
(0.22710537910461426, '고객님께 보내드리겠습니다 또 이용해주세요 제발 감사합니다 ')

Best_reply > 고객님께서 이 땡기실 때마다 저희가 생각나도록 더욱 열심히 하겠습니다아 주문 정말 감사드리고 다음에 또 봐요 
Best_similarity_score: 0.31089699268341064
--------------------------------------------------------------------------------------------
user > 피자가 너무 맛있어요
(0.2719748020172119, '고객님들 칭찬덕에 매장 운영에 큰 힘을 얻고 최선을 다하고 있습니다. 감사합니다. 다음에 또 주문해주세요 ')
(0.2719748020172119, '고객님들 칭찬덕에 매장 운영에 큰 힘을 얻고 최선을 다하고 있습니다. 감사합니다. 다음에 또 주문해주세요 ')
(0.2719748020172119, '고객님들 칭찬덕에 매장 운영에 큰 힘을 얻고 최선을 다하고 있습니다. 감사합니다. 다음에 또 주문해주세요 ')

Best_reply > 고객님들 칭찬덕에 매장 운영에 큰 힘을 얻고 최선을 다하고 있습니다. 감사합니다. 다음에 또 주문해주세요 
Best_similarity_score: 0.2719748020172119
--------------------------------------------------------------------------------------------
user > 밥이 너무 식어서 왔어요
(0.21604180335998535, '고객님들의 감사함에 보답

KeyboardInterrupt: ignored

In [ ]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}gpt_22000_0920").to(device)
# 0908_gpt_freeze
chatbot_with_evaluation(loaded_model,tokenizer,512,device)


In [ ]:

max_len = 2028


def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=3) :
    model.eval()
    while True:
        review = input("user > ").strip()
        if review == "quit":
            break

        review = review.replace('<itore>','')
        review = review.replace('<menu>','')
        review = review.replace('<region>','')

        text = "<q>" + review + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=5.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=1.15,
                                        top_k=60,
                                        num_beams=2,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=5,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)

            generated_text = generated_text.replace('<itore>','')
            generated_text = generated_text.replace('<menu>','')
            generated_text = generated_text.replace('<region>','')


            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)

        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [ ]:
chatbot_with_evaluation(loaded_model,tokenizer,1024,device)

In [ ]:
positive_reviews = [
    "오늘 배달 시간이 빨라서 좋았어요. 음식도 맛있게 먹었습니다!",
    "음식이 신선하고 양도 푸짐해서 만족스러웠어요.",
    "서비스가 친절하고 음식이 정말 맛있었어요. 다음에도 주문할게요!",
    "배달원 분이 너무 친절하셨어요. 감사합니다!",
    "배달이 빨라서 기분이 좋았습니다. 음식도 예상대로 맛있었어요.",
    "가격 대비 음식의 품질이 좋아서 좋았어요.",
    "주문한 음식이 따뜻하게 도착했고 맛있게 먹었습니다.",
    "매번 여기서 주문하는데 항상 만족스러워요. 최고에요!",
    "음식이 정말 맛있어서 가족 모두가 만족했어요.",
    "주문한 음식이 신선하게 잘 도착했고, 배달원 분이 친절해서 기분 좋았습니다."
]

negative_reviews = [
    "음식이 덜 익어서 먹을 수가 없었어요. 실망했습니다.",
    "배달이 매우 늦어져서 기다리는 동안 짜증났어요.",
    "음식 양도 적고 가격 대비 별로였습니다.",
    "주문한 음식이 차가워서 더 이상 시킬 의향이 없습니다.",
    "배달원 분이 불친절해서 좋지 않은 경험이었습니다.",
    "음식이 짠 맛이 나서 먹을 수가 없었습니다. 돈이 아까웠어요.",
    "음식이 퍼져서 배달 상태가 좋지 않았습니다.",
    "음식 포장이 엉망이었고, 음식 퀄리티도 별로여서 실망스러웠습니다.",
    "음식이 별로였고, 가격도 비싸서 다른 곳에서 시킬게요.",
    "배달이 완전히 틀려서 다시 기다리느라 시간 낭비했습니다."
]

test_reviews = positive_reviews + negative_reviews

In [ ]:
positive_reviews_messed_up = [
    "오늘배달시간이빨라서좋았어요.음식도맛있게먹었습니다!",
    "음식이완전쩔어서양도개많아서만족해짐.",
    "서비스가엄청친절하고음식이진짜맛있었어요.다음에도진짜주문할게요!",
    "배달원분이개쌍판데기친절하셨어요.감사합니다!",
    "배달이엄청빨라서기분이쩐다.음식도완전예상대로맛있었어요.",
    "가격대비음식의퀄리티가미쳤어서좋았어요.",
    "주문한음식이엄청따뜻하게도착했고맛있게먹었어요.",
    "매번여기서주문하는데항상개쩐다.진짜최고에요!",
    "음식이정말미친맛이어서가족모두가개좋아했어요.",
    "주문한음식이엄청신선했고,배달원분이개친절해서개기분좋았습니다."
]